# Anticipez les besoins en consommations de bâtiments

## Objectif

L'objectif de ce projet est de prédire les émissions de CO2 et la consommation totale d'énergie de bâtiments non destinés à l'habitation de la ville de Seattle à partir de leurs données structurelles (taille, usage, date de construction, localisation) afin d'atteindre l'objectif de neutralité carbone de la ville en 2050.

## 1. Installation du projet et de son environnement

Afin d’utiliser correctement ce notebook, vérifiez que vous disposez du bon environnement pour l’exécuter.


### 1.1 Installation de Python

Pour ce projet, il est nécessaire d’avoir **au minimum Python 3.8**.  
Si ce n’est pas déjà le cas, vous pouvez vous référer à [la documentation officielle](https://www.python.org/downloads/).

Vérifiez votre version de Python :

```bash
python --version
```

### 1.2 Installation de `uv`

`uv` est un gestionnaire de projets Python permettant d’installer et d’organiser les dépendances plus rapidement et plus simplement que les outils traditionnels (`pip`, `virtualenv`, etc.).

Pour installer `uv`, veuillez suivre [la documentation officielle](https://docs.astral.sh/uv/getting-started/installation/#standalone-installer)

Vérifiez l’installation :

```bash
uv --version
```

### 1.3 Création du projet

Créez un nouveau projet Python avec `uv` :

```bash
uv init nom_du_projet
cd nom_du_projet
```

La structure de base du projet est alors générée automatiquement.


### 1.4 Création et activation de l’environnement virtuel

Créez l’environnement virtuel :

```bash
uv venv
```

Activez-le selon votre système :

* **Linux / macOS**

```bash
source .venv/bin/activate
```

* **Windows (PowerShell)**

```powershell
.venv\Scripts\Activate.ps1
```

Voir la documentation officielle :
[https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment](https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment)



### 1.5 Installation des dépendances

> ❗ Assurez-vous que l’environnement virtuel est **activé** avant d’installer les dépendances.

Installez les bibliothèques nécessaires au projet :

```bash
uv add ipykernel jupyterlab seaborn matplotlib numpy pandas scikit-learn tqdm scipy
```

Ces dépendances sont automatiquement enregistrées dans le fichier `pyproject.toml`.

Vous pouvez visualiser l’ensemble des dépendances installées avec la commande suivante :

```bash
uv tree


## 2. Importation des Bibliothèques


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.axes import Axes
import seaborn as sns
import numpy as np
from scipy import stats
from typing import List, Union

In [ ]:
def distribution_column(df: pd.DataFrame, column: str):
    """
    Affiche la distribution de la colonne 'Outlier' dans le DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame contenant la colonne 'Outlier'.
    """
    print(f"📊 Distribution de la colonne {column}")
    print("-" * 100)
    
    outlier_counts = df[column].value_counts(dropna=False)
    outlier_pct = df[column].value_counts(dropna=False, normalize=True) * 100
    
    outlier_summary = pd.DataFrame({
        'Effectif': outlier_counts,
        'Pourcentage': outlier_pct.round(2)
    })
    
    display(outlier_summary)

def display_columns_info(df: pd.DataFrame, columns: List[str]):
    for col in columns:
        n_unique = df[col].nunique()
        n_missing = df[col].isna().sum()
        pct_unique = (n_unique / len(df)) * 100
        pct_missing = (n_missing / len(df)) * 100
        
        print(f"• {col}:")
        print(f"  - Type : {df[col].dtype}")
        print(f"  - Valeurs uniques: {n_unique} ({pct_unique:.1f}%)")
        print(f"  - Valeurs manquantes: {n_missing} ({pct_missing:.1f}%)")
        
        # Afficher les valeurs si peu nombreuses
        if n_unique <= 10:
            print(f"  - Valeurs: {df[col].unique()}")
        print("")

def remove_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Supprime les colonnes spécifiées du DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame d'origine.
        columns (list): Liste des noms de colonnes à supprimer.

    Returns:
        pd.DataFrame: Le DataFrame sans les colonnes supprimées.
    """
    print("📦 Suppression de colonnes")
    print(f"   ➤ Shape initiale        : {df.shape}")
    print(f"   ➤ Colonnes demandées    : {len(columns)}")

    # Colonnes réellement présentes
    existing_cols = [col for col in columns if col in df.columns]
    missing_cols = [col for col in columns if col not in df.columns]

    df = df.drop(columns=existing_cols)

    print(f"✅ Suppression terminée")
    print(f"   ➤ Shape finale          : {df.shape}")
    print(f"   ➤ Colonnes supprimées   : {len(existing_cols)}")

    if existing_cols:
        print(f"   🧹 Liste supprimée       : {existing_cols}")

    if missing_cols:
        print(f"   ⚠️ Colonnes inexistantes : {missing_cols}")

    return df

def compare_describe_statistics(
    df_ref,
    df_comp,
    columns,
    stats=('mean', '50%', 'std'),
    stat_labels=('Moyenne', 'Médiane', 'Écart-type'),
    ref_label='Avec outliers',
    comp_label='Sans outliers',
    n_cols=2,
    figsize=(16, 12),
    title="Comparaison des statistiques descriptives",
    summary_columns=None,
    mean_alert_thresholds=(5, 10)
):
    """
    Compare les statistiques issues de df.describe() entre deux dataframes.

    Parameters
    ----------
    df_ref : pd.DataFrame
        DataFrame de référence (ex : avec outliers)

    df_comp : pd.DataFrame
        DataFrame comparé (ex : sans outliers)

    columns : list
        Colonnes numériques à comparer

    stats : tuple
        Statistiques issues de describe() à comparer

    stat_labels : tuple
        Labels affichés pour les statistiques

    ref_label / comp_label : str
        Libellés affichés dans les graphiques

    n_cols : int
        Nombre de colonnes dans la grille matplotlib

    figsize : tuple
        Taille de la figure

    title : str
        Titre global

    summary_columns : list or None
        Colonnes incluses dans le résumé textuel

    mean_alert_thresholds : tuple
        Seuils (%) pour interpréter l'impact sur la moyenne
    """

    desc_ref = df_ref[columns].describe()
    desc_comp = df_comp[columns].describe()

    n_rows = int(np.ceil(len(columns) / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()

    fig.suptitle(title, fontsize=16, fontweight='bold', y=0.995)

    for idx, col in enumerate(columns):
        ax = axes[idx]

        ref_values = [desc_ref.loc[s, col] for s in stats]
        comp_values = [desc_comp.loc[s, col] for s in stats]

        variations = [
            ((comp - ref) / ref * 100) if ref != 0 else np.nan
            for ref, comp in zip(ref_values, comp_values)
        ]

        x = np.arange(len(stats))
        width = 0.35

        ax.bar(
            x - width / 2,
            ref_values,
            width,
            label=ref_label,
            color='#e74c3c',
            alpha=0.8
        )

        bars_comp = ax.bar(
            x + width / 2,
            comp_values,
            width,
            label=comp_label,
            color='#2ecc71',
            alpha=0.8
        )

        ax.set_title(col, fontweight='bold')
        ax.set_ylabel('Valeur')
        ax.set_xticks(x)
        ax.set_xticklabels(stat_labels)
        ax.legend()
        ax.grid(axis='y', linestyle='--', alpha=0.3)

        # Annotation variation %
        for bar, delta in zip(bars_comp, variations):
            height = bar.get_height()
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height,
                f"{delta:+.1f}%",
                ha='center',
                va='bottom',
                fontsize=9,
                fontweight='bold',
                color='green' if delta < 0 else 'red'
            )

    # Supprimer axes inutilisés
    for i in range(len(columns), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

    # ======================
    # Résumé textuel
    # ======================
    if summary_columns is None:
        summary_columns = columns

    print("\n" + "=" * 80)
    print("RÉSUMÉ DE LA COMPARAISON DES STATISTIQUES")
    print("=" * 80)

    mod_threshold, high_threshold = mean_alert_thresholds

    for col in summary_columns:
        ref_mean = desc_ref.loc['mean', col]
        comp_mean = desc_comp.loc['mean', col]
        delta_mean = ((comp_mean - ref_mean) / ref_mean * 100) if ref_mean != 0 else np.nan

        print(f"\n{col} :")
        for s, label in zip(stats, stat_labels):
            ref_val = desc_ref.loc[s, col]
            comp_val = desc_comp.loc[s, col]
            delta = ((comp_val - ref_val) / ref_val * 100) if ref_val != 0 else np.nan

            print(f"   • {label} : {delta:+.2f}%")

        if abs(delta_mean) > high_threshold:
            print(f"   ⚠️ Impact FORT sur la moyenne (>{high_threshold}%)")
        elif abs(delta_mean) > mod_threshold:
            print(f"   ⚡ Impact MODÉRÉ sur la moyenne")
        else:
            print(f"   ✅ Impact FAIBLE sur la moyenne")

## 3. Chargement du jeu de donnée

- **Jeu de données** : [2016_Building_Energy_Benchmarking.csv](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P4/2016_Building_Energy_Benchmarking.csv)
- **Source officielle** : [Building Energy Benchmarking - Seattle Open Data](https://data.seattle.gov/Built-Environment/Building-Energy-Benchmarking-Data-2015-Present/teqw-tu6e/about_data)

> 📁 **Action** : Téléchargez le jeu de données et ajoutez-le dans le dossier `data/`

In [ ]:
# Configuration de l'affichage pandas
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.width', None)         # Largeur automatique

df_origin = pd.read_csv('data/2016_Building_Energy_Benchmarking.csv')

df = df_origin.copy()

## 4. Choix des variables cibles

Pour répondre à l'objectif, je dois choisir 2 variables cibles. L'une pour prédire les émissions de CO2 et l'autre pour la consommation totale d'énergie.

### Sélection des variables cibles

Après analyse du dataset, je choisis les 2 variables cibles suivantes :

**1. Pour les émissions de CO2 :** `TotalGHGEmissions`
- Émissions totales de gaz à effet de serre (en tonnes métriques de CO2)

**2. Pour la consommation d'énergie :** `SiteEnergyUse(kBtu)`
- Consommation totale d'énergie du site (en kBtu)

#### Alternatives non retenues

**Pour les émissions de CO2 :**
- `GHGEmissionsIntensity` : Mesure d'intensité (émissions par surface en sq ft), moins pertinente pour l'objectif global de neutralité carbone de la ville qui nécessite les émissions totales absolues

**Pour la consommation d'énergie :**
- `SiteEnergyUseWN(kBtu)` : Version normalisée par la météo, moins représentative de la consommation réelle
- `SourceEUI(kBtu/sf)` : Mesure d'intensité énergétique (par surface), pas la consommation totale
- `SiteEUI(kBtu/sf)` : Intensité d'usage du site par surface, ne reflète pas la consommation totale
- `Electricity(kWh)`, `NaturalGas(therms)`, `SteamUse(kBtu)` : Mesures partielles par type d'énergie, ne représentent pas la consommation totale agrégée

In [ ]:
TARGET_CO2 = 'TotalGHGEmissions'
TARGET_ENERGY = 'SiteEnergyUse(kBtu)'

## 5. Information générale du Dataset

In [ ]:
df.head()

In [ ]:
display(df.shape)

**Dimensions du dataset :**
- 3376 lignes (bâtiments)
- 46 colonnes

In [ ]:
display(df.info())

#### Synthèse de la structure des données

**Types de données :**
- 8 colonnes numériques entières (`int64`)
- 22 colonnes numériques décimales (`float64`)
- 15 colonnes textuelles (`object`)
- 1 colonne booléenne (`bool`)

**Colonnes à analyser pouvant contenir des lignes problématiques :**
- `Outlier` : indique si un bâtiment est considéré comme un outlier
- `ComplianceStatus` : indique si un bâtiment a satisfait aux exigences réglementaires de déclaration énergétique

**Valeurs manquantes importantes :**
- `Comments` : 0 valeur renseignée (colonne complètement vide)
- `YearsENERGYSTARCertified` : 119 / 3376 (3,5 %)
- `Outlier` : 32 / 3376 (0,9 %)
- `ThirdLargestPropertyUseType(GFA)` : 596 / 3376 (17,7 %)
- `SecondLargestPropertyUseType(GFA)` : 1679 / 3376 (49,7 %)
- `ENERGYSTARScore` : 2533 / 3376 (75,0 %)

**Variables cibles : couverture excellente**
- `TotalGHGEmissions` : 3367 / 3376 (99,7 %)
- `SiteEnergyUse(kBtu)` : 3371 / 3376 (99,9 %)

**Points d'attention :**
- `ZipCode` : stocké en `float64` au lieu de `string` (16 valeurs manquantes)



In [ ]:
df.describe()

### Synthèse des statistiques descriptives

**Colonnes analysées :** 22 colonnes numériques (sur 46 au total)

**Observations principales :**

1. **Caractéristiques des bâtiments :**
   - Année de construction : entre 1900 et 2015 (médiane : 1975)
   - Nombre d'étages : 0 à 99 (médiane : 4 étages)
   - Surface totale (PropertyGFATotal) : très variable de 11,285 à 9,320,156 sq ft

2. **Valeurs aberrantes détectées :**
   - `Electricity(kWh)` : valeur minimale négative (-33,827)
   - `Electricity(kBtu)` : valeur minimale négative (-115,417)
   - `TotalGHGEmissions` : valeur minimale négative (-0.8)
   - `GHGEmissionsIntensity` : valeur minimale négative (-0.02)
   - `SourceEUIWN(kBtu/sf)` : valeur minimale négative (-2.1)

3. **Distribution des variables cibles :**
   - **TotalGHGEmissions :** moyenne = 119.7, médiane = 33.9 (distribution asymétrique)
   - **SiteEnergyUse(kBtu) :** moyenne = 5.4M, médiane = 1.8M (forte variabilité)

4. **Forte variabilité :**
   - Les écarts-types sont très élevés pour la plupart des variables énergétiques
   - Présence probable de valeurs extrêmes (outliers)
   - Exemple : SteamUse varie de 0 à 134M kBtu

5. **Score ENERGYSTAR :**
   - Moyenne : 67.9/100 (sur 2533 bâtiments notés)
   - 75% des bâtiments notés ont un score ≥ 53

In [ ]:
def analyze_missing_values(df: pd.DataFrame, display_table: bool = True) -> pd.DataFrame:
    """
    Analyse les valeurs manquantes dans un DataFrame.
    
    Args:
        df (pd.DataFrame): Le DataFrame à analyser.
        display_table (bool): Si True, affiche le tableau détaillé des valeurs manquantes.
    
    Returns:
        pd.DataFrame: DataFrame avec les statistiques de valeurs manquantes par colonne.
    """
    # 1. Pourcentage global de cellules vides
    total_cells = df.shape[0] * df.shape[1]
    missing_cells = df.isna().sum().sum()
    pct_missing_global = (missing_cells / total_cells) * 100
    
    print(f"\n🌐 Pourcentage de cellules vides sur tout le DataFrame : {pct_missing_global:.2f}%")
    
    # 2. Pourcentage par colonne
    missing_by_column = df.isna().sum()
    pct_by_column = (missing_by_column / len(df)) * 100
    
    # Créer un DataFrame pour les statistiques
    missing_df = pd.DataFrame({
        'Colonne': df.columns,
        'Valeurs manquantes': missing_by_column.values,
        'Pourcentage (%)': pct_by_column.values
    })
    
    # Trier par pourcentage décroissant
    missing_df = missing_df.sort_values('Pourcentage (%)', ascending=False)
    
    # Afficher le tableau si demandé
    # if display_table:
    #     print(f"\n📋 Pourcentage de cellules vides par colonne :")
    #     print("-" * 80)
    #     display(missing_df[missing_df['Pourcentage (%)'] > 0])

    return missing_df

def display_missing_values(df: pd.DataFrame):
    """
    Analyse les valeurs manquantes dans un DataFrame.
    
    Args:
        df (pd.DataFrame): Le DataFrame à analyser.
        display_table (bool): Si True, affiche le tableau détaillé des valeurs manquantes.
    
    Returns:
        pd.DataFrame: DataFrame avec les statistiques de valeurs manquantes par colonne.
    """
    print(f"\n📋 Pourcentage de cellules vides par colonne :")
    print("-" * 80)
    missing_stats = analyze_missing_values(df, True)
    display(missing_stats[missing_stats['Pourcentage (%)'] > 0])


def plot_missing_values(missing_df: pd.DataFrame, top_n: int = 15, min_threshold: float = 0.1):
    """
    Visualise les valeurs manquantes sous forme de graphique à barres horizontales.
    
    Args:
        missing_df (pd.DataFrame): DataFrame retourné par analyze_missing_values().
        top_n (int): Nombre maximum de colonnes à afficher (par défaut: 15).
        min_threshold (float): Pourcentage minimum pour afficher une colonne (par défaut: 0.1%).
    """
    # Filtrer les colonnes selon le seuil
    missing_cols = missing_df[missing_df['Pourcentage (%)'] >= min_threshold].head(top_n).copy()
    
    if len(missing_cols) > 0:
        fig, ax = plt.subplots(figsize=(14, max(6, len(missing_cols) * 0.4)))
        
        # Créer le graphique horizontal
        bars = ax.barh(range(len(missing_cols)), missing_cols['Pourcentage (%)'])
        ax.set_yticks(range(len(missing_cols)))
        ax.set_yticklabels(missing_cols['Colonne'], fontsize=10)
        ax.set_xlabel('Pourcentage de valeurs manquantes (%)', fontsize=12, fontweight='bold')
        ax.set_title('Principales colonnes avec valeurs manquantes', fontsize=14, fontweight='bold', pad=20)
        ax.grid(axis='x', alpha=0.3, linestyle='--')
        
        # Colorer les barres selon le niveau de gravité
        colors = ['#2ecc71' if x < 1 else '#f39c12' if x < 5 else '#e74c3c' 
                  for x in missing_cols['Pourcentage (%)']]
        for bar, color in zip(bars, colors):
            bar.set_color(color)
        
        # Ajouter les valeurs sur les barres
        for i, (idx, row) in enumerate(missing_cols.iterrows()):
            ax.text(row['Pourcentage (%)'] + 0.5, i, f"{row['Pourcentage (%)']:.2f}%", 
                    va='center', fontsize=9, fontweight='bold')
        
        legend_elements = [
            Patch(facecolor='#2ecc71', label='< 1% manquant (excellente couverture)'),
            Patch(facecolor='#f39c12', label='1-5% manquant (bonne couverture)'),
            Patch(facecolor='#e74c3c', label='> 5% manquant (attention requise)')
        ]
        ax.legend(handles=legend_elements, loc='upper right', fontsize=9)
        
        # Ajuster les marges pour éviter les warnings
        plt.subplots_adjust(left=0.25, right=0.95, top=0.95, bottom=0.1)
        plt.show()
    else:
        print(f"✅ Aucune colonne avec ≥ {min_threshold}% de valeurs manquantes !")


In [ ]:
missing_stats = analyze_missing_values(df, True)
plot_missing_values(missing_stats, top_n=15, min_threshold=0.1)

## 6. Filtrage des bâtiments non résidentiels

Pour le projet, nous avons uniquement besoin des bâtiments non résidentiels. Pour cela, nous allons commencer par filtrer le jeu de données.

In [ ]:
df["BuildingType"].value_counts()

**Observations :**

- **NonResidential (1 460)** : Bâtiments non résidentiels  
- **Multifamily LR (1–4) (1 018)** : Immeubles multifamiliaux à faible densité  
- **Multifamily MR (5–9) (580)** : Immeubles multifamiliaux à densité moyenne  
- **Multifamily HR (10+) (110)** : Immeubles multifamiliaux à haute densité  
- **SPS-District K–12 (98)** : Établissements scolaires publics (de la maternelle au lycée)  
- **Nonresidential COS (85)** : Bâtiments municipaux de la ville de Seattle  
- **Campus (24)** : Ensembles de bâtiments interconnectés  
- **Nonresidential WA (1)** : Bâtiment appartenant à l’État de Washington  

Dans le cadre du projet, seuls les bâtiments **non destinés à l’habitation** sont requis. Par conséquent, nous conserverons uniquement les catégories suivantes :

- **NonResidential (1 460)** : Bâtiments non résidentiels  
- **SPS-District K–12 (98)** : Établissements scolaires publics  
- **Nonresidential COS (85)** : Bâtiments municipaux de Seattle  
- **Campus (24)** : Ensembles de bâtiments interconnectés
- **Nonresidential WA (1)**: Bâtiment de l’État de Washington


In [ ]:
print(f"Nombre de lignes avant filtrage par type de batiment : {len(df)}")

# Types de bâtiments à conserver
building_types_to_keep = [
    "NonResidential",
    "SPS-District K-12",
    "Nonresidential COS",
    "Campus",
    "Nonresidential WA"
]

# Filtrer les données
df = df[df['BuildingType'].isin(building_types_to_keep)]

# Documenter le nombre de lignes après filtrage
print(f"Nombre de lignes après filtrage : {len(df)}")
print(f"Nombre de lignes supprimées : {len(df_origin) - len(df)}")
# Afficher la répartition des types de bâtiments conservés
print("\nRépartition des types de bâtiments conservés :")
print(df['BuildingType'].value_counts())

## 7. Vérification des doublons

La vérification des doublons est essentielle pour garantir la qualité des données avant toute analyse ou modélisation. Des doublons peuvent fausser les statistiques, introduire des biais dans les modèles prédictifs et conduire à des conclusions erronées.

### Stratégie de détection

Je vais vérifier les doublons selon plusieurs niveaux de granularité :

1. **Doublons complets** : Lignes identiques sur toutes les colonnes (très rare)
2. **Doublons sur l'identifiant** : `OSEBuildingID` (doit être unique par définition)
3. **Doublons géographiques** : Combinaison `Address + City + ZipCode`

### Pourquoi plusieurs vérifications ?

- Un même bâtiment pourrait avoir des données légèrement différentes (erreurs de saisie, mises à jour)
- Plusieurs bâtiments pourraient partager une même adresse (complexes immobiliers)

In [ ]:
def verify_duplicates(
    df: pd.DataFrame,
    columns: Union[List[str], None] = None,
    display_table: bool = False,
    preview_rows: int = 10
) -> int:
    """
    Vérifie les doublons dans un DataFrame.

    Args:
        df (pd.DataFrame): DataFrame à analyser
        columns (list[str] | None): Colonnes à utiliser pour la détection.
        display_table (bool): Si True, affiche les lignes dupliquées
        preview_rows (int): Nombre de lignes dupliquées à afficher

    Returns:
        int: Nombre de doublons détectés
    """
    if df.empty:
        print("⚠️ Le DataFrame est vide.")
        return 0

    subset = None
    if columns:
        missing_cols = set(columns) - set(df.columns)
        if missing_cols:
            raise ValueError(f"Colonnes inexistantes : {missing_cols}")
        subset = columns
        label = f"Doublons sur {columns}"
    else:
        label = "Toutes les colonnes"

    duplicates = df.duplicated(subset=subset).sum()
    percentage = (duplicates / len(df)) * 100

    print(f"\n Colonnes concernés : {label} ")
    print(f"\n🔍 Nombre de doublons : {duplicates}")
    print(f"📊 Pourcentage : {percentage:.2f}%")

    if duplicates > 0 and display_table:
        print("\n🔍 Aperçu des lignes dupliquées :")
        print("-" * 80)

        sort_cols = subset if subset else df.columns.tolist()
        duplicated_rows = (
            df[df.duplicated(subset=subset, keep=False)]
            .sort_values(by=sort_cols)
        )

        try:
            display(duplicated_rows.head(preview_rows))
        except NameError:
            print(duplicated_rows.head(preview_rows))

    return duplicates

# Exécution des vérifications
print(f"📊 ANALYSE DES DOUBLONS")
print(f"="*80)

# 1. Vérifier le nombre total de lignes dupliquées (toutes colonnes confondues)
print("\nVérification des doublons complets (toutes colonnes)")
total_duplicates = verify_duplicates(df)

# 2. Vérifier les doublons sur l'identifiant unique du bâtiment
print("\nVérification sur l'identifiant OSEBuildingID")
id_duplicates = verify_duplicates(df, ['OSEBuildingID'])

# 3. Vérifier les doublons sur l'adresse
print("\nVérification sur l'adresse")
address_duplicates = verify_duplicates(df, ['Address', 'City', 'ZipCode'])

In [ ]:
print("\nVérification sur l'adresse + nom de la propriété")
address_duplicates = verify_duplicates(df, ['PropertyName', 'Address', 'City', 'ZipCode'])

In [ ]:
print("\n Vérification avec le rajout de la longitude et latitude")
address_duplicates = verify_duplicates(df, ['Latitude', 'Longitude', 'Address', 'City', 'ZipCode'])

In [ ]:
print("\nVérification avec le rajout du type de bâtiment")
address_duplicates = verify_duplicates(df, ['BuildingType', 'Latitude', 'Longitude', 'Address', 'City', 'ZipCode'])

In [ ]:
print("\nVérification avec le rajout de l'année de construction")
address_duplicates = verify_duplicates(df, ['BuildingType', 'YearBuilt', 'Latitude', 'Longitude', 'Address', 'City', 'ZipCode'], True)

In [ ]:
print("\nVérification avec le rajout du type d'utilisation principale")
address_duplicates = verify_duplicates(df, ['BuildingType', 'LargestPropertyUseType', 'YearBuilt', 'Latitude', 'Longitude', 'Address', 'City', 'ZipCode'], True)

In [ ]:
# 4. Vérification finale : doublons sur les variables cibles
print("\nVérification sur les variables cibles (consommations énergétiques identiques)")
print("-" * 80)
print("\nCette vérification détecte les bâtiments ayant exactement les mêmes valeurs")
print("de consommation énergétique ET d'émissions de CO2.")
print("Un tel cas serait hautement suspect et pourrait indiquer un vrai doublon.\n")

# Vérifier sur les deux variables cibles simultanément
target_duplicates = verify_duplicates(
    df,
    [TARGET_ENERGY, TARGET_CO2],
    display_table=True,
    preview_rows=10
)

### Conclusion sur l'analyse des doublons

Après avoir effectué une analyse exhaustive des doublons selon différents critères, voici les résultats :

#### Résultats des vérifications

| Critère de vérification | Doublons détectés | Statut |
|------------------------|-------------------|--------|
| **Lignes totalement identiques** | 0 | ✅ OK |
| **Identifiant unique (`OSEBuildingID`)** | 0 | ✅ OK |
| **Adresse géographique** (`Address + City + ZipCode`) | 20 | ⚠️ À analyser |
| **Adresse + Nom du bâtiment** | 0 | ✅ OK |
| **Coordonnées GPS + Adresse** | 19 | ⚠️ À analyser |
| **+ Type de bâtiment + Année** | 7 | ⚠️ À analyser |
| **+ Type d'utilisation principale** | 3 | ⚠️ À analyser |
| **Variables cibles identiques** | 5 | ⚠️ Valeurs manquantes |

#### Interprétation

**Aucun vrai doublon détecté** ✅

##### 1. Doublons géographiques (21 cas)

Les 20 "doublons" détectés sur l'adresse géographique correspondent à des **bâtiments distincts partageant la même adresse**, ce qui est légitime dans le cas de :

- **Complexes immobiliers** : Plusieurs bâtiments au sein d'un même parc d'affaires
  - Exemple : *Cloverdale Business Park* (bâtiments B, C, E)
  - Exemple : *Airport Way Center* (bâtiments C, D)

**Preuves de distinction** :
- Chaque ligne a un **`OSEBuildingID` unique** (identifiant officiel)
- Chaque bâtiment a un **`PropertyName` différent**
- Les **surfaces** (`PropertyGFATotal`) sont différentes
- Les **consommations énergétiques** sont différentes
- Les **émissions de CO2** sont différentes

##### 2. Doublons sur variables cibles (10 cas)

Les 5 "doublons" détectés sur les variables cibles (`SiteEnergyUse(kBtu)` et `TotalGHGEmissions`) correspondent en réalité à des **bâtiments sans données énergétiques**.

**Explication** : Ces bâtiments partagent les mêmes valeurs de variables cibles car elles sont toutes **manquantes (NaN)**. La fonction `duplicated()` de pandas considère que deux valeurs NaN sont identiques.

**Nature du problème** : Il ne s'agit pas de doublons mais de **valeurs manquantes** qui devront être traitées lors de l'étape de nettoyage des données.

#### Décision

**Aucune suppression de lignes n'est nécessaire.** 

- Toutes les observations représentent des bâtiments uniques et légitimes
- Les bâtiments sans données énergétiques seront traités lors de la gestion des valeurs manquantes

Le dataset est prêt pour l'analyse exploratoire et la modélisation sans risque de biais lié aux doublons.

## 8. Vérification des colonnes pouvant contenir des lignes problématiques


### 8.1 La colonne `Outlier`

In [ ]:
distribution_column(df, 'Outlier')

**Observation :**

La variable *Outlier* comporte trois états :
- **Low outlier** : valeur anormalement basse détectée ;
- **High outlier** : valeur anormalement élevée ;
- **Valeur manquante** : aucune anomalie identifiée.

On constate que les bâtiments identifiés comme *outliers* ne représentent que **0,92 %** du jeu de données.

In [ ]:
# Afficher uniquement les Low Outliers
colonnes_a_afficher = ['BuildingType', 'PrimaryPropertyType', 'DefaultData', 'ComplianceStatus', 'ENERGYSTARScore', 'Outlier', TARGET_CO2, TARGET_ENERGY]

low_outliers_stats = df[df['Outlier'] == 'Low outlier']
print("Statistiques pour Low Outliers:")
display(low_outliers_stats[colonnes_a_afficher].head(15))



**Observations des Low Outliers (15 bâtiments - 0.90%)** :
- **ENERGYSTARScore** : valeurs très élevées (99, 100) ou manquantes (NaN)
- **ComplianceStatus** : principalement "Non-Compliant" ou "Error - Correct Default Data"
- **Interprétation** : ces bâtiments ont une consommation d'énergie anormalement **faible** malgré leur non-conformité, ce qui suggère des données potentiellement erronées ou incomplètes

In [ ]:
# Afficher uniquement les High Outliers
high_outliers_stats = df[df['Outlier'] == 'High outlier']
print("Statistiques pour High Outliers:")
display(high_outliers_stats[colonnes_a_afficher].head())

**Observations sur les High Outliers (2 bâtiments - 0.12%)** :
- **ENERGYSTARScore** : valeurs très faibles (1) ou manquantes (NaN)
- **ComplianceStatus** : "Non-Compliant"
- **Interprétation** : ces bâtiments ont une consommation d'énergie anormalement **élevée** et sont également non-conformes, ce qui est cohérent avec leur mauvaise performance énergétique

**Synthèse** : Les outliers (17 bâtiments au total, soit 1.02% du dataset) présentent systématiquement des problèmes de conformité et des scores ENERGY STAR extrêmes ou absents, renforçant l'hypothèse de problèmes de qualité des données.

#### Identification de la Méthode de Détection

In [ ]:
# Tester différentes métriques avec la méthode IQR
for metric in [TARGET_ENERGY, TARGET_CO2, 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'GHGEmissionsIntensity']:
    # 1. Calcul IQR
    Q1 = df[metric].quantile(0.25)
    Q3 = df[metric].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # 2. Calcul Z-score
    z_scores = stats.zscore(df[metric].dropna())
    abs_z_scores = np.abs(z_scores)

    # Identification des outliers
    outliers_z2 = df[metric].dropna()[abs_z_scores > 2]
    outliers_iqr = df[(df[metric] < lower) | (df[metric] > upper)]

    # Calculer le rappel (recall) pour les outliers détectés
    high_detected = df[(df[metric] > upper) & (df['Outlier'] == 'High outlier')]
    low_detected = df[(df[metric] < lower) & (df['Outlier'] == 'Low outlier')]
    print(f"\nMétrique: {metric}")
    print(f"Nombre de high outliers détectés via IQR: {len(outliers_iqr[outliers_iqr['Outlier'] == 'High outlier'])}")
    print(f"Nombre de low outliers détectés via IQR: {len(outliers_iqr[outliers_iqr['Outlier'] == 'Low outlier'])}")

    print(f"Nombre de high outliers détectés via z-score (seuil = 2): {len(outliers_z2[df['Outlier'] == 'High outlier'])}")
    print(f"Nombre de low outliers détectés via z-score(seuil = 2): {len(outliers_z2[df['Outlier'] == 'Low outlier'])}")

**Observation :**

Les tests de détection d'outliers révèlent que **ni la méthode IQR ni le Z-score ne permettent d'identifier les outliers marqués dans la colonne**.

Résultats :
- **Méthode IQR** : détecte seulement 1-2 bâtiments parmi les 17 outliers marqués sur les métriques cibles
- **Z-score (seuil = 2 )** :  détecte seulement 1 outliers sur la métrique ENERGYSTARScore

**Conclusion :** La colonne `Outlier` ne provient probablement **pas d'une méthode statistique classique**, mais plutôt :
- D'une **règle métier spécifique** liée aux exigences réglementaires
- D'une **validation manuelle** par les autorités de Seattle
- D'une **détection d'anomalies contextuelles** (ex: données incohérentes avec le type de bâtiment)

Cette hypothèse est renforcée par le fait que **100% des outliers sont non-conformes ou en erreur**.

In [ ]:

from typing import List, Optional, Tuple

def create_boxplot_by_category(df: pd.DataFrame,
                               column: str,
                               category_column: str,
                               ax: plt.Axes,
                               title: str,
                               ylabel: str,
                               xlabel: str = "Catégorie d'outlier",
                               log_scale: bool = True,
                               exclude_non_positive: bool = True) -> None:
    """
    Crée un boxplot d'une variable par catégorie avec options de mise en forme.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame source
    column : str
        Nom de la colonne à visualiser
    category_column : str
        Nom de la colonne de catégories
    ax : plt.Axes
        Axes matplotlib sur lequel dessiner
    title : str
        Titre du graphique
    ylabel : str
        Label de l'axe Y
    xlabel : str
        Label de l'axe X
    log_scale : bool
        Utiliser une échelle logarithmique
    exclude_non_positive : bool
        Exclure les valeurs <= 0
    """
    # Filtrer les données si nécessaire
    if exclude_non_positive:
        df_plot = df[df[column] > 0].copy()
    else:
        df_plot = df.copy()
    
    # Créer le boxplot
    df_plot.boxplot(column=column, by=category_column, ax=ax)
    
    # Appliquer l'échelle log si demandé
    if log_scale:
        ax.set_yscale('log')
    
    # Mise en forme
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.get_figure().suptitle('')  # Supprimer le titre automatique


def visualize_variables_by_outlier_category(df: pd.DataFrame,
                                            variables: List[dict],
                                            category_column: str = 'Outlier_Category',
                                            figsize: Tuple[int, int] = (16, 6),
                                            main_title: str = "Distribution des variables par catégorie d'outlier") -> None:
    """
    Crée une visualisation multi-graphiques des variables par catégorie d'outlier.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame source
    variables : List[dict]
        Liste de dictionnaires contenant les paramètres pour chaque variable:
        - 'column': nom de la colonne
        - 'title': titre du graphique
        - 'ylabel': label de l'axe Y
        - 'xlabel': (optionnel) label de l'axe X
        - 'log_scale': (optionnel) utiliser échelle log
        - 'exclude_non_positive': (optionnel) exclure valeurs <= 0
    category_column : str
        Nom de la colonne de catégories
    figsize : Tuple[int, int]
        Taille de la figure
    main_title : str
        Titre principal de la figure
        
    Example:
    --------
    variables = [
        {
            'column': 'TotalGHGEmissions',
            'title': 'Émissions de GHG (échelle log)',
            'ylabel': 'TotalGHGEmissions (tonnes CO₂)',
            'log_scale': True
        },
        {
            'column': 'SiteEnergyUse',
            'title': 'Consommation énergétique (échelle log)',
            'ylabel': 'SiteEnergyUse (kBtu)',
            'log_scale': True
        }
    ]
    visualize_variables_by_outlier_category(df, variables)
    """
    n_vars = len(variables)
    fig, axes = plt.subplots(1, n_vars, figsize=figsize)
    fig.suptitle(main_title, fontsize=14, fontweight='bold')
    
    # S'assurer que axes est toujours une liste
    if n_vars == 1:
        axes = [axes]
    
    for i, var_params in enumerate(variables):
        # Paramètres par défaut
        params = {
            'xlabel': "Catégorie d'outlier",
            'log_scale': True,
            'exclude_non_positive': True
        }
        # Mettre à jour avec les paramètres fournis
        params.update(var_params)
        
        create_boxplot_by_category(
            df=df,
            column=params['column'],
            category_column=category_column,
            ax=axes[i],
            title=params['title'],
            ylabel=params['ylabel'],
            xlabel=params['xlabel'],
            log_scale=params['log_scale'],
            exclude_non_positive=params['exclude_non_positive']
        )
    
    plt.tight_layout()
    plt.show()

df_outlier = df.copy()
df_outlier['Outlier_Category'] = df_outlier['Outlier'].fillna('No outlier')

variables = [
    {
        'column': TARGET_CO2,
        'title': 'Émissions de GHG (échelle log)',
        'ylabel': 'TotalGHGEmissions (tonnes CO₂)'
    },
    {
        'column': TARGET_ENERGY,
        'title': 'Consommation énergétique (échelle log)',
        'ylabel': 'SiteEnergyUse (kBtu)'
    },
    {
        'column': 'SourceEUI(kBtu/sf)',
        'title': 'Source EUI (échelle log)',
        'ylabel': 'SourceEUI (kBtu/sf)'
    },
    {
        'column': 'SiteEUI(kBtu/sf)',
        'title': 'Site EUI (échelle log)',
        'ylabel': 'SiteEUI (kBtu/sf)'
    },
    {
        'column': 'GHGEmissionsIntensity',
        'title': 'GHGEmissionsIntensity',
        'ylabel': 'GHGEmissionsIntensity'
    }
]

visualize_variables_by_outlier_category(
    df=df_outlier,
    variables=variables,
    figsize=(18, 6)
)

**Observations** :
- Les catégories "No outlier" présentent systématiquement les dispersions les plus importantes sur tous les indicateurs
- Les "High outlier" ont des valeurs élevées mais concentrées
- Les "Low outlier" montrent généralement des valeurs faibles avec peu de dispersion
- L'échelle logarithmique révèle des différences de 3 à 4 ordres de grandeur entre les bâtiments
- Une forte corrélation semble exister entre les différents indicateurs énergétiques

#### Conclusion et décision de traitement des outliers

Les bâtiments identifiés comme *outliers* représentent une part très faible du jeu de données (17 bâtiments, soit environ **1 %**). Cependant, ils présentent tous des **problèmes de qualité des données**.

Ces bâtiments sont majoritairement **non conformes** ou associés à des **erreurs de déclaration**, avec des **scores ENERGY STAR extrêmes ou manquants**. Les méthodes statistiques classiques (IQR, Z-score) ne permettent pas de les détecter, ce qui indique que la variable `Outlier` repose sur des **règles métier ou réglementaires**, et non sur une logique statistique exploitable en modélisation.

De plus, leurs distributions sont peu dispersées et atypiques par rapport au reste du dataset, ce qui pourrait **biaiser les modèles prédictifs**.

#### Décision retenue
- **Suppression de la colonne `Outlier`**
- **Exclusion des 17 bâtiments outliers du dataset de modélisation**

Cette décision est justifiée par :
- des **données non fiables**
- un **risque de biais élevé**
- un **impact négligeable sur la taille et la représentativité** du jeu de données
- la volonté de **privilégier la qualité des données** pour la modélisation


In [ ]:
# Supprimer les lignes marquées comme outliers (High outlier ou Low outlier)
# Garder uniquement les lignes où Outlier est NaN (pas d'outlier)
df_clean = df[df['Outlier'].isna()].copy()

# Afficher le résultat
print(f"📊 Résultat du nettoyage:")
print(f"   • Taille initiale      : {len(df):,} lignes")
print(f"   • Taille après nettoyage : {len(df_clean):,} lignes")
print(f"   • Lignes supprimées    : {len(df) - len(df_clean)} lignes ({(len(df) - len(df_clean))/len(df)*100:.2f}%)")

df = df_clean

In [ ]:
df = remove_columns(df, ['Outlier'])

### 8.2. La colonne `ComplianceStatus`

In [ ]:
def analyze_missing(df: pd.DataFrame, column: str, target: str):
    total = len(df[target]);
    n_value = df[column].value_counts()

    # Valeurs manquantes
    missing = df[target].isna().groupby(df[column]).sum()

    # Valeurs nulles (égales à 0)
    zero = (df[target] == 0).groupby(df[column]).sum()

    result_df = pd.DataFrame({
        'Total': n_value,
        'Total (%)': (n_value / total * 100).round(2),
        'Valeurs manquantes': missing,
        'Valeurs manquantes (%)': (missing / n_value * 100).round(2),
        'Valeurs nulles': zero,
        'Valeurs nulles (%)': (zero / n_value * 100).round(2)
    })

    return result_df

def plot_distribution(df: pd.DataFrame, column: str, xlabel, ylabel, targets: List[str] = []):
    """
    Affiche un graphique de distribution pour la colonne de compliance.
    
    :param df: DataFrame pandas contenant les données
    :param column: Nom de la colonne à analyser
    :param xlabel: Label de l'axe des abscisses
    :param ylabel: Label de l'axe des ordonnées
    """
    plt.figure(figsize=(10, 5))
    
    compliance_counts = df[column].value_counts()

    
    ax = compliance_counts.plot(kind='bar', color='steelblue')
    
    plt.title(f'Distribution de la colonne {column}', fontsize=14, fontweight='bold')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha='right')
    
    total = len(df)
    for i, v in enumerate(compliance_counts):
        ax.text(i, v + 0.01 * total, f'{v}\n({v/total*100:.1f}%)', ha='center')

    if(targets):
        for i, target in enumerate(targets):
            analyze_missing_target = analyze_missing(df, column, target)
            ax = compliance_counts.plot(kind='bar')

    
    plt.tight_layout()
    plt.show()

display_columns_info(df, ['ComplianceStatus'])
plot_distribution(df, 'ComplianceStatus', 'Statut de conformité', 'Nombre de bâtiments', [TARGET_CO2, TARGET_ENERGY])

**Observation** :

La colonne `ComplianceStatus` indique si un bâtiment a satisfait aux exigences réglementaires de déclaration énergétique.  
Elle contient **4 valeurs** :

- **Compliant (93.8 %, 1548 lignes)** : Données correctement déclarées et conformes  
- **Error - Correct Default Data (5.2 %, 86 lignes)** : Erreurs détectées, données par défaut corrigées  
- **Non-Compliant (0.2 %, 3 lignes)** : Non-conformité réglementaire  
- **Missing Data (0.8 %, 14 lignes)** : Données manquantes lors de la déclaration  

##### Analysons les données 'Missing Data'

In [ ]:
display(df[df['ComplianceStatus'] == 'Missing Data'].head(10))

**Observation** :

Les 14 lignes *Missing Data* (principalement des écoles K-12) présentent les caractéristiques suivantes :

- **`SiteEnergyUse(kBtu)`** : Systématiquement à **0.0** (donnée absente ou non reportée)
- **`DefaultData`** : Toujours **False** (pas de correction appliquée)
- **Consommation énergétique partielle** : Seule l'électricité **OU** le gaz naturel est renseigné, jamais les deux
  - Exemples : Électricité uniquement (1.5-2.3 M kBtu) avec NaturalGas = 0.0
  - Ou inversement : NaturalGas uniquement (3.2-4.3 M kBtu) avec Electricity = 0.0
- **`TotalGHGEmissions`** : Calculable et cohérent (4.19 à 229.38) basé sur la source d'énergie disponible

**Décision** : Ces lignes contiennent des **données partielles exploitables**. Représente que 0.8% du jeu de données, on supprime les lignes

##### Analysons les données 'Non-Compliant'

In [ ]:
display(df[df['ComplianceStatus'] == 'Non-Compliant'].head(10))

**Observation** :

Les 3 lignes *Non-Compliant* (bâtiments non-résidentiels anciens : 1924-1990) sont **totalement inutilisables** :

- **`SiteEnergyUse(kBtu)`** : Soit 0.0, soit NaN (aucune donnée valide)
- **`TotalGHGEmissions`** : Soit 0.0, soit NaN (impossible à calculer)
- **`GHGEmissionsIntensity`** : Soit 0.0, soit NaN
- **`DefaultData`** : False (aucune correction n'a été appliquée)
- **Colonnes énergétiques** : 2/3 des lignes présentent des NaN complets

**Conclusion** : Ces lignes ne contiennent aucune information exploitable pour la prédiction des émissions ou de la consommation énergétique.

**Décision** : **Supprimer ces 3 lignes** du dataset

##### Analysons les données 'Error - Correct Default Data'

In [ ]:
df_compliance_issues_error = df[df['ComplianceStatus'] == 'Error - Correct Default Data'].copy()
display(df_compliance_issues_error.head())

In [ ]:
distribution_column(df_compliance_issues_error, 'DefaultData')

columns_to_compare = [
    TARGET_ENERGY,
    TARGET_CO2
]
df_without_compliance_errors = df[df["ComplianceStatus"] != "Error - Correct Default Data"].copy()

**Observation** :

Les 86 lignes *Error - Correct Default Data* (majoritairement des écoles K-12) présentent des **données de haute qualité après correction** :

- **`DefaultData`** : Systématiquement **True** (données corrigées automatiquement)
- **Double source énergétique** : Électricité **ET** Gaz naturel renseignés (pas de données partielles)
  - Electricity : 0.9 à 7.8 M kBtu
  - NaturalGas : 0.0 à 5.7 M kBtu (incluant cas "électrique seulement")
- **`SiteEUI(kBtu/sf)`** : Valeurs cohérentes (25.6 - 53.3)
- **`TotalGHGEmissions`** : Calculées correctement (10.55 à 359.09)
- **`GHGEmissionsIntensity`** : Plage normale (0.19 à 1.82)
- **Aucune valeur NaN** dans les variables énergétiques et cibles

**Conclusion** : Bien que marquées "Error" initialement, ces lignes semble avoir été **corrigées avec succès** et contiennent des données complètes et fiables. Nous pouvons concervés ces lignes

In [ ]:
def remove_lignes(df: pd.DataFrame, condition: str) -> pd.DataFrame:
    """
    Supprime les lignes du DataFrame selon une condition donnée.

    Args:
        df (pd.DataFrame): Le DataFrame d'origine.
        condition (str): La condition pour filtrer les lignes à supprimer.

    Returns:
        pd.DataFrame: Le DataFrame sans les lignes supprimées.
    """
    print("📦 Suppression de lignes selon une condition")
    print(f"   ➤ Shape initiale        : {df.shape}")

    initial_count = len(df)
    df = df.query(f"not ({condition})")
    final_count = len(df)

    print(f"✅ Suppression terminée")
    print(f"   ➤ Shape finale          : {df.shape}")
    print(f"   ➤ Lignes supprimées     : {initial_count - final_count} lignes")

    return df

df = remove_lignes(df, "ComplianceStatus == 'Non-Compliant'")
df = remove_lignes(df, "ComplianceStatus == 'Missing Data'")
distribution_column(df, 'ComplianceStatus')

## 9. Vérification des colonnes avec des données manquantes 

Avant de construire un modèle prédictif, il est nécessaire d'éliminer les colonnes qui n'apportent pas d'information utile ou qui contiennent trop de valeurs manquantes.




### 9.1. Colonnes avec taux élevé de valeurs manquantes (> 50%)

**Colonnes concernées :**
- `Comments` (100% manquant)
- `YearsENERGYSTARCertified` (94.1% manquant)
- `SecondLargestPropertyUseType` (48.7% manquant)
- `SecondLargestPropertyUseTypeGFA` (48.7% manquant)
- `ThirdLargestPropertyUseType` (78.8% manquant)
- `ThirdLargestPropertyUseTypeGFA` (78.8% manquant)

**Analysons chaque colonne :**


In [ ]:
display_columns_info(df, ['Comments', 'YearsENERGYSTARCertified', 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA'])

#### Décision de suppression

**À supprimer :**

1. **`Comments`** : Colonne entièrement vide (100% manquant)
   - *Aucune description disponible dans le dataset*
   - Aucune information exploitable

2. **`YearsENERGYSTARCertified`** : 94.1% manquant
   - *Liste des années de certification ENERGY STAR*
   - Information marginale, peu de bâtiments certifiés

3. **`SecondLargestPropertyUseType`** : 48.7% manquant
   - *Deuxième type d'utilisation le plus important basé sur la surface*
   - Information secondaire déjà capturée par `LargestPropertyUseType`

4. **`SecondLargestPropertyUseTypeGFA`** : 48.7% manquant
   - *Surface du deuxième type d'utilisation le plus important*
   - Même raison, information redondante

5. **`ThirdLargestPropertyUseType`** : 78.8% manquant
   - *Troisième type d'utilisation le plus important basé sur la surface*
   - Information tertiaire trop peu disponible et peu pertinente

6. **`ThirdLargestPropertyUseTypeGFA`** : 78.8% manquant
   - *Surface du troisième type d'utilisation le plus important*
   - Même raison, trop de valeurs manquantes


In [ ]:
colonnes_a_supprimer = [
    'Comments',
	'YearsENERGYSTARCertified',
	'SecondLargestPropertyUseType',
	'SecondLargestPropertyUseTypeGFA',
	'ThirdLargestPropertyUseType',
	'ThirdLargestPropertyUseTypeGFA'
]

df = remove_columns(df, colonnes_a_supprimer)

### 9.2. Colonnes > 20 % de donnée manquante

**Colonnes concernées :**
- `ENERGYSTARScore` (33.72% manquant) : Note calculée par l'EPA de 1 à 100 évaluant la performance énergétique globale d'une propriété, basée sur des données nationales pour contrôler les différences de climat, d'utilisations des bâtiments et d'opérations. Un score de 50 représente la médiane nationale.

**Analysons la colonne :**

In [ ]:
display_columns_info(df, ['ENERGYSTARScore'])

#### Décision sur ENERGYSTARScore

**Choix retenu : Suppression de la colonne**

**Justification :**

1. **Score non recalculable** : Le score ENERGYSTAR est calculé par l'EPA selon des modèles propriétaires et des données nationales de référence non disponibles dans notre dataset.

2. **Taux élevé de valeurs manquantes** : 25% des bâtiments (843/3376) n'ont pas de score, ce qui limiterait significativement l'utilité prédictive de cette variable.

3. **Risque de biais** : Toute imputation (moyenne, médiane, régression) introduirait un biais important dans les prédictions.

**Conclusion :** La suppression de cette colonne évite d'introduire du biais dans nos modèles prédictifs tout en conservant les variables énergétiques réellement mesurées qui sont nos vraies variables d'intérêt.


In [ ]:
df = remove_columns(df, ['ENERGYSTARScore'])

### 9.3. Colonnes > 1 % de donnée manquante

#### La colonne ZipCode

In [ ]:
display_columns_info(df, ['ZipCode'])

**Observation** :
- Il y a 48 valeurs unique dont 16 (1%) valeurs manquantes
- `ZipCode` est stocké en float64 et devrait être une variable catégorielle (string/object)


Nous pouvons surement récupérer les ZipCode manquante en ce basant sur les `Neighborhood`. Vérifions la colonne.

In [ ]:
display_columns_info(df, ['Neighborhood'])
distribution_column(df, 'Neighborhood')

Il n’y a aucune donnée manquante, mais la colonne doit être nettoyée car elle contient des doublons :
- **BALLARD** (majuscules) et **Ballard** (minuscules)
- Même problème pour **DELRIDGE NEIGHBORHOODS** et **Delridge**
- Même problème pour **CENTRAL** et **Central**


In [ ]:
df["Neighborhood"] = df["Neighborhood"].str.upper()
df["Neighborhood"] = df["Neighborhood"].replace(
    "DELRIDGE NEIGHBORHOODS",
    "DELRIDGE"
)

distribution_column(df, 'Neighborhood')

On peut comparés ZipCode et Neighborhood

In [ ]:
def calculate_mapping_stats(grouped_series):
    """
    Calcule les statistiques de mapping pour une série groupée.
    
    Returns:
    --------
    dict avec les stats incluant les cas 0, 1, et multiples correspondances
    """
    zero_mapping = (grouped_series == 0).sum()
    one_to_one = (grouped_series == 1).sum()
    multiple = (grouped_series > 1).sum()
    total = len(grouped_series)
    
    return {
        'zero': zero_mapping,
        'zero_pct': zero_mapping / total * 100 if total > 0 else 0,
        'one_to_one': one_to_one,
        'one_to_one_pct': one_to_one / total * 100 if total > 0 else 0,
        'multiple': multiple,
        'multiple_pct': multiple / total * 100 if total > 0 else 0,
        'total': total
    }


def print_mapping_stats(stats, from_label, to_label):
    """
    Affiche les statistiques de mapping de manière formatée.
    """
    print(f"\n📊 Mapping {from_label} → {to_label} :")
    
    if stats['zero'] > 0:
        print(f"   • {from_label} sans {to_label} : {stats['zero']}/{stats['total']} ({stats['zero_pct']:.1f}%)")
    
    print(f"   • {from_label} correspondant à 1 seul {to_label} : {stats['one_to_one']}/{stats['total']} ({stats['one_to_one_pct']:.1f}%)")
    print(f"   • {from_label} correspondant à plusieurs {to_label} : {stats['multiple']}/{stats['total']} ({stats['multiple_pct']:.1f}%)")


def count_unique_mappings(df, col1, col2, ascending=False):
    """
    Compte le nombre de valeurs uniques de col2 pour chaque valeur de col1.
    
    Parameters:
    -----------
    df : DataFrame
    col1 : str
        Colonne de regroupement
    col2 : str
        Colonne dont on compte les valeurs uniques
    ascending : bool, default False
        Ordre de tri
    
    Returns:
    --------
    Series : Nombre de valeurs uniques de col2 par col1, triée
    """
    return df.groupby(col1)[col2].nunique().sort_values(ascending=ascending)

In [ ]:
print("="*80)
print("ANALYSE ZipCode vs Neighborhood")
print("="*80)

# Utilisation
zip_to_neighborhood = count_unique_mappings(df, 'ZipCode', 'Neighborhood')
neighborhood_to_zip = count_unique_mappings(df, 'Neighborhood', 'ZipCode')

print("\n📊 Nombre de quartiers (Neighborhood) par code postal (ZipCode) :")
print(zip_to_neighborhood.head(10))

print("\n📊 Nombre de codes postaux (ZipCode) par quartier (Neighborhood) :")
print(neighborhood_to_zip)
# 4. Mapping ZipCode → Neighborhood
stats_zip = calculate_mapping_stats(zip_to_neighborhood)
print_mapping_stats(stats_zip, 'ZipCodes', 'Neighborhood')

# 5. Mapping inverse Neighborhood → ZipCode
stats_neighborhood = calculate_mapping_stats(neighborhood_to_zip)
print_mapping_stats(stats_neighborhood, 'Neighborhoods', 'ZipCode')

**Solution d'imputation** : Utiliser le **ZipCode le plus fréquent** dans chaque quartier pour imputer les valeurs manquantes

In [ ]:
def get_zipcode_most_frequency(df: pd.DataFrame, column: str):
    results = []
    for neighborhood in df[column].unique():
        # Bâtiments avec ZipCode dans le même quartier
        with_zip_in_n = df[
            (df[column] == neighborhood) &
            (df['ZipCode'].notna())
        ]
        
        if len(with_zip_in_n) > 0:
            # ZipCode le plus fréquent
            most_common = with_zip_in_n['ZipCode'].mode()[0]
            frequency_pct = round(
                (with_zip_in_n['ZipCode'] == most_common).sum()
                / len(with_zip_in_n)
                * 100,
                2
            )
        else:
            most_common = None
            frequency_pct = None
        
        results.append({
            'Neighborhood': neighborhood,
            'ZipCode': most_common,
            'FrequencyPct': frequency_pct
        })
    return pd.DataFrame(results)
zipcode_summary_df = get_zipcode_most_frequency(df, 'Neighborhood')
display(zipcode_summary_df)

In [ ]:
neighborhood_to_zip = (
    zipcode_summary_df
    .dropna(subset=['ZipCode'])
    .set_index('Neighborhood')['ZipCode']
)

df['ZipCode'] = df['ZipCode'].fillna(
    df['Neighborhood'].map(neighborhood_to_zip)
)



In [ ]:
df["ZipCode"] = df["ZipCode"].astype("Int64").astype(str)

In [ ]:
display_columns_info(df, ['ZipCode'])

In [ ]:
missing_stats = analyze_missing_values(df, True)
display(missing_stats)

#### Colonne LargestPropertyUseType

In [ ]:
display(df[df['LargestPropertyUseType'].isna()].head())

Les colonnes suivantes sont identifiées comme candidates potentielles pour l’imputation :

- `PrimaryPropertyType`
- `ListOfAllPropertyUseTypes`

In [ ]:
def correspondance_columns(df: pd.DataFrame, columns_to_ref: str, columns_to_compare: str):
    """
    Effectue une correspondance entre deux colonnes catégorielles et affiche les résultats.
    """
    df_comparison = df[df[columns_to_ref].notna() & df[columns_to_compare].notna()].copy()
    df_comparison['same_value'] = df_comparison[columns_to_ref] == df_comparison[columns_to_compare]

    print(f"Pourcentage de correspondance : {df_comparison['same_value'].mean() * 100:.2f}%")

correspondance_columns(df_origin, 'LargestPropertyUseType', 'PrimaryPropertyType')
df_copy = df_origin.copy()
df_copy["FirstPropertyUseType"] = (
    df_copy["ListOfAllPropertyUseTypes"]
    .str.split(",")
    .str[0]
    .str.strip()
)
correspondance_columns(df_copy, 'LargestPropertyUseType', 'FirstPropertyUseType')


`ListOfAllPropertyUseTypes` présente un taux de correspondance élevé ; cette colonne sera donc utilisée pour l’imputation.

In [ ]:
df["FirstPropertyUseType"] = (
    df["ListOfAllPropertyUseTypes"]
    .str.split(",")
    .str[0]
    .str.strip()
)

print("\n🔧 Imputation de LargestPropertyUseType...")
na_count_before = df['LargestPropertyUseType'].isnull().sum()

# Imputer avec PrimaryPropertyType
df['LargestPropertyUseType'] = df['LargestPropertyUseType'].fillna(df['FirstPropertyUseType'])

na_count_after = df['LargestPropertyUseType'].isnull().sum()
print(f"  ✅ {na_count_before - na_count_after} valeurs imputées")
if na_count_after > 0:
    print(f"  ⚠️  {na_count_after} valeurs manquantes restantes")

remove_columns(df, ['FirstPropertyUseType'])

#### Colonne LargestPropertyUseTypeGFA

In [ ]:
display(df[df['LargestPropertyUseTypeGFA'].isna()].head())

In [ ]:
df_largest_missing_values = df_origin.copy()
cols_secondary = [
    'SecondLargestPropertyUseTypeGFA',
    'ThirdLargestPropertyUseTypeGFA'
]

df_largest_missing_values["TMP_LargestPropertyUseTypeGFA"] = (
    df_largest_missing_values['PropertyGFATotal']
    - df_largest_missing_values[cols_secondary].sum(axis=1, skipna=True)
)

df_largest_missing_values["TMP_PropertyGFATotal"] = (
    df_largest_missing_values[
        ['LargestPropertyUseTypeGFA'] + cols_secondary
    ].sum(axis=1, skipna=True)
)

display(df_largest_missing_values[["PropertyGFATotal", "TMP_PropertyGFATotal", "LargestPropertyUseTypeGFA", "TMP_LargestPropertyUseTypeGFA", "SecondLargestPropertyUseTypeGFA", "ThirdLargestPropertyUseTypeGFA"]].head())

correspondance_columns(df_largest_missing_values, 'LargestPropertyUseTypeGFA', 'TMP_LargestPropertyUseTypeGFA')
correspondance_columns(df_largest_missing_values, 'PropertyGFATotal', 'TMP_PropertyGFATotal')

**Observation:** 
- 3 des 4 bâtiments ont un **usage unique**  (voir ListOfAllPropertyUseTypes).
- Presque 50 % de correspondance

**Solution:** Imputer avec `PropertyGFATotal`

In [ ]:
print("\n🔧 Imputation de LargestPropertyUseType...")
na_count_before = df['LargestPropertyUseTypeGFA'].isnull().sum()

# Imputer avec PropertyGFATotal
df['LargestPropertyUseTypeGFA'] = df['LargestPropertyUseTypeGFA'].fillna(df['PropertyGFATotal'])

na_count_after = df['LargestPropertyUseTypeGFA'].isnull().sum()
print(f"  ✅ {na_count_before - na_count_after} valeurs imputées")
if na_count_after > 0:
    print(f"  ⚠️  {na_count_after} valeurs manquantes restantes")

#### Colonne SiteEUIWN(kBtu/sf) et SiteEnergyUseWN(kBtu)

In [ ]:
display(df[df['SiteEUIWN(kBtu/sf)'].isna()].head())

**Observation:** 
- Une seule ligne avec valeurs manquantes (index 563)
- La ligne possède `SiteEnergyUse(kBtu)` = 5177270.5
- `SiteEnergyUseWN(kBtu)` est la version normalisée par météo de `SiteEnergyUse(kBtu)`

**Solution:** 
1. Calculer le ratio moyen entre `SiteEnergyUseWN` et `SiteEnergyUse`
2. Appliquer ce ratio pour imputer `SiteEnergyUseWN(kBtu)`
3. Calculer `SiteEUIWN(kBtu/sf)` à partir de `SiteEnergyUseWN(kBtu)` et la surface

In [ ]:
print("\n🔧 Imputation de SiteEnergyUseWN(kBtu) et SiteEUIWN(kBtu/sf)...")

# 1. Calculer le ratio moyen entre SiteEnergyUseWN et SiteEnergyUse
ratio = df['SiteEnergyUseWN(kBtu)'] / df['SiteEnergyUse(kBtu)']
mean_ratio = ratio.mean()
print(f"  📊 Ratio moyen SiteEnergyUseWN/SiteEnergyUse : {mean_ratio:.4f}")

# 2. Imputer SiteEnergyUseWN(kBtu) avec le ratio moyen
na_mask = df['SiteEnergyUseWN(kBtu)'].isna()
na_count_before = na_mask.sum()

df.loc[na_mask, 'SiteEnergyUseWN(kBtu)'] = df.loc[na_mask, 'SiteEnergyUse(kBtu)'] * mean_ratio

na_count_after = df['SiteEnergyUseWN(kBtu)'].isna().sum()
print(f"  ✅ SiteEnergyUseWN(kBtu) : {na_count_before - na_count_after} valeur(s) imputée(s)")

# 3. Calculer SiteEUIWN(kBtu/sf) = SiteEnergyUseWN(kBtu) / PropertyGFABuilding(s)
na_mask_eui = df['SiteEUIWN(kBtu/sf)'].isna()
na_count_eui_before = na_mask_eui.sum()

df.loc[na_mask_eui, 'SiteEUIWN(kBtu/sf)'] = (
    df.loc[na_mask_eui, 'SiteEnergyUseWN(kBtu)'] / df.loc[na_mask_eui, 'PropertyGFABuilding(s)']
)

na_count_eui_after = df['SiteEUIWN(kBtu/sf)'].isna().sum()
print(f"  ✅ SiteEUIWN(kBtu/sf) : {na_count_eui_before - na_count_eui_after} valeur(s) imputée(s)")

In [ ]:
display_missing_values(df)

## 10 Détection et traitement des valeurs aberrantes

### 10.1 Valeurs négatives

In [ ]:
def display_columns_with_negatives(dataframe: pd.DataFrame) -> None:
    """
    Affiche les statistiques descriptives des colonnes numériques contenant des valeurs négatives.

    Parameters
    ----------
    dataframe : pandas.DataFrame
        Le DataFrame à analyser
    """
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns
    cols_with_negatives = numeric_cols[(dataframe[numeric_cols] < 0).any()]
    display(dataframe[cols_with_negatives].describe())

display_columns_with_negatives(df)

**Observation** :
On remarque que nous avons des valeurs négatives pour SourceEUIWN(kBtu/sf), Electricity(kWh), Electricity(kBtu), TotalGHGEmissions, GHGEmissionsIntensity cela signifierai que les batiments produirés plus de d'électricité par exemple que n'en produit.

Vérifions les lignes concernés

In [ ]:
negative_checks = [
    ('SourceEUIWN(kBtu/sf)', df['SourceEUIWN(kBtu/sf)'] < 0),
    ('Electricity(kWh)', df['Electricity(kWh)'] < 0),
    ('Electricity(kBtu)', df['Electricity(kBtu)'] < 0),
    ('TotalGHGEmissions', df['TotalGHGEmissions'] < 0),
    ('GHGEmissionsIntensity', df['GHGEmissionsIntensity'] < 0),
]
display(df[df['Electricity(kWh)'] < 0])

On constate que nous avons qu'un seul batiment possédant des valeurs négatives

Le bâtiment **Bullitt Center** est un bâtiment écologique, possédant des panneaux solaires, ce qui pourrait expliquer ces valeurs négatives.

Nous décidons de supprimer cette donnée car :
- Il s’agit d’un cas très spécifique de bâtiment à énergie positive (unique dans le dataset) ;
- Valeurs incohérentes : consommation électrique négative alors que la consommation totale est positive (SiteEnergyUse (kBtu)) ;
- Donnée non représentative de la population cible (bâtiments consommateurs d’énergie) ;
- Risque de fausser les modèles de prédiction ;
- On constate également que les colonnes **TotalGHGEmissions** et **GHGEmissionsIntensity** comportent des valeurs négatives.

In [ ]:
df = remove_lignes(df, "`Electricity(kWh)` < 0")

In [ ]:
display_columns_with_negatives(df)

**Observation**
Il n'y a plus de valeur négative

### 10.2 Valeurs à zéro

In [ ]:
def display_columns_with_zero(dataframe: pd.DataFrame) -> None:
    """
    Affiche les statistiques descriptives des colonnes numériques contenant des valeurs négatives.

    Parameters
    ----------
    dataframe : pandas.DataFrame
        Le DataFrame à analyser
    """
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns
    cols_with_zero = numeric_cols[(dataframe[numeric_cols] == 0).any()]
    display(dataframe[cols_with_zero].describe())

display_columns_with_zero(df)

**Observation** :  
On remarque que certaines valeurs sont à zéro pour les colonnes relatives à la consommation d'énergie ou aux émissions de CO₂, ainsi que pour des informations concernant les bâtiments, le parking et le nombre de bâtiments.

Vérifions les lignes concernées afin de nous assurer que ces valeurs à zéro sont justifiées.


#### Commençons par les variables cibles

In [ ]:
zero_value_checks = (
    (df[TARGET_CO2] == 0) |
    (df[TARGET_ENERGY] == 0)
)

display(df[zero_value_checks])

**Observations**

L'analyse des lignes ayant des valeurs cibles (`TotalGHGEmissions` et `SiteEnergyUse(kBtu)`) à zéro révèle **2 bâtiments problématiques** :

- **Olympic Hills Elementary (Ligne 1361)** - École K-12
    - **Problème majeur** : Toutes les métriques énergétiques sont à zéro
    - `DefaultData = True` : Utilisation de données par défaut/placeholder
    - `ComplianceStatus = "Error - Correct Default Data"` : Erreur confirmée nécessitant correction

- **Whole Foods Interbay (Ligne 513)** - Services personnels
    - **Incohérence** : `SiteEnergyUse(kBtu) = 12,525,174.0` (consommation élevée) mais `TotalGHGEmissions = 0.0`
    - Toutes les composantes énergétiques (Electricity, NaturalGas, etc.) sont à 0.0
    - `DefaultData = False` et `ComplianceStatus = Compliant`
    - **Hypothèse** : Possible erreur de calcul des émissions ou utilisation d'énergie 100% renouvelable non comptabilisée

**Décision**
Ces 2 lignes présentent des **problèmes de qualité de données** et devraient être **exclues** du jeu d'entraînement pour éviter de biaiser les modèles de prédiction des émissions de CO2 et de consommation énergétique.

In [ ]:
df = remove_lignes(df, f"`{TARGET_CO2}` == 0 or `{TARGET_ENERGY}` == 0")

#### Colonne SiteEnergyUse(kBtu)

In [ ]:
display(df[df['SteamUse(kBtu)'] == 0].head())

**Observation** :
La colonne *SteamUse (kBtu)* indique la consommation annuelle de vapeur. Tous les bâtiments ne sont pas raccordés au réseau de vapeur.

**Décision** : On conserve la donnée.


#### Colonnes Electricity(kWh) et Electricity(kBtu)

In [ ]:
zero_value_checks = (
    (df['Electricity(kWh)'] == 0) |
    (df['Electricity(kBtu)'] == 0)
)

display(df[zero_value_checks])

**Observation** :  
Nous n’avons qu’une seule ligne. Un restaurant ne peut pas fonctionner sans électricité (éclairage, réfrigération, ventilation). Ici, la totalité de l’énergie provient du gaz, ce qui correspond à un scénario hautement improbable.

**Décision** :  
Si ce restaurant n’utilise réellement pas d’électricité, il s’agit d’un cas spécifique ; dans le cas contraire, c’est une erreur. Nous choisissons donc de supprimer cette ligne afin d’éviter tout biais.


In [ ]:
df = remove_lignes(df, f"`Electricity(kWh)` == 0 or `Electricity(kBtu)` == 0")

#### Colonnes NaturalGas(therms) et NaturalGas(kBtu)

In [ ]:
zero_value_checks = (
    (df['NaturalGas(therms)'] == 0) |
    (df['NaturalGas(kBtu)'] == 0)
)

display(df[zero_value_checks].head(2))

**Observation** :
Aujourd'hui beaucoup de bâtiments sont 100 % électriques., nous pouvons conserver les zéros

#### Colonnes PropertyGFAParking 

In [ ]:
display(df[df['PropertyGFAParking'] == 0].head(2))

**Observation** :  
Certains bâtiments n’ont aucune surface de parking (ex. bureaux urbains, établissements publics anciens).

Nous pouvons vérifier que la donnée est correcte, car :  
*PropertyGFATotal = PropertyGFAParking + PropertyGFABuilding(s)*.



In [ ]:
df_parking_zero = df[df['PropertyGFAParking'] == 0].copy()
df_parking_zero['Property_Calculed'] = df_parking_zero['PropertyGFAParking'] + df_parking_zero['PropertyGFABuilding(s)']

correspondance_columns(df_parking_zero, 'PropertyGFATotal', 'Property_Calculed')

**Décision**: on conserve la donnée

#### Colonne NumberofBuildings

In [ ]:
print("\n🔧 Nombre de batiments avec 0 bâtiment : {}".format(df[df['NumberofBuildings'] == 0].shape[0]))
display(df[df['NumberofBuildings'] == 0].head(2))

In [ ]:
median_building_by_type = df.groupby('PrimaryPropertyType')['NumberofBuildings'].median()
display(median_building_by_type)
distribution_column(df[df['NumberofBuildings'] == 0], 'BuildingType')


**Décision** : On peut imputer une valeur de 1, car la médiane pour les bâtiments de ce type est de 1. Et on en profite pour mettre le bon type à la donnée Int64

In [ ]:
print("\n🔧 Imputation de NumberofBuildings...")
na_count_before = len(df[df['NumberofBuildings'] == 0])
# Imputer avec mediane par type de propriété
df.loc[df['NumberofBuildings'] == 0, 'NumberofBuildings'] = 1

na_count_after = len(df[df['NumberofBuildings'] == 0])
print(f"  ✅ {na_count_before - na_count_after} valeurs imputées")
if na_count_after > 0:
    print(f"  ⚠️  {na_count_after} valeurs manquantes restantes")



df["NumberofBuildings"] = df["NumberofBuildings"].astype("Int64")


#### Colonne NumberofFloors

In [ ]:
print("\n🔧 Nombre de batiments avec 0 étage : {}".format(df[df['NumberofFloors'] == 0].shape[0]))
display(df[df['NumberofFloors'] == 0].head(2))

**Observation**:
Tout bâtiment a au moins 1 étage, nous allons l'imputé via la médianne par type de propriété

In [ ]:
median_floors_by_type = df.groupby('PrimaryPropertyType')['NumberofFloors'].median()

df.loc[df['NumberofFloors'] == 0, 'NumberofFloors'] = df.loc[df['NumberofFloors'] == 0, 'PrimaryPropertyType'].map(median_floors_by_type)



#### Colonne SourceEUI(kBtu/sf)

In [ ]:

var = 'SourceEUI(kBtu/sf)'

mask_zero = df[var] == 0

display(df[mask_zero].head(2))

ratios = (
    df
    .assign(ratio=df[var] / df['SiteEUI(kBtu/sf)'])
    .query("ratio > 0")
    .groupby('PrimaryPropertyType')['ratio']
    .median()
)

df.loc[mask_zero, var] = (
    df.loc[mask_zero, 'PrimaryPropertyType']
      .map(ratios)
      * df.loc[mask_zero, 'SiteEUI(kBtu/sf)']
)




#### Colonne SourceEUIWN(kBtu/sf)

In [ ]:
var = 'SourceEUIWN(kBtu/sf)'
mask_zero = df[var] == 0
n_zeros_initial = mask_zero.sum()

if n_zeros_initial > 0:
    print(f"   Zéros détectés: {n_zeros_initial}")
    print(f"   Méthode: Imputation par SourceEUI(kBtu/sf)")

    # Imputer par SourceEUI (qui a maintenant été imputé lui-même)
    df.loc[mask_zero, var] = df.loc[mask_zero, 'SourceEUI(kBtu/sf)']

    print(f"   ✅ {n_zeros_initial} valeur(s) imputée(s) par SourceEUI")
else:
    print(f"   ✅ Aucun zéro détecté")

#### Colonne SiteEUIWN(kBtu/sf)

In [ ]:
df_siteEUIWN = df[df['SiteEUIWN(kBtu/sf)'] == 0].copy()
print(f"\n🔧 Nombre de batiments : {df_siteEUIWN.shape[0]}")

print(len(df_siteEUIWN[df_siteEUIWN['SiteEnergyUseWN(kBtu)'] == 0]))
display(df_siteEUIWN.head())

In [ ]:
df.loc[df['SiteEUIWN(kBtu/sf)'] == 0, 'SiteEUIWN(kBtu/sf)'] = \
    df.loc[df['SiteEUIWN(kBtu/sf)'] == 0, 'SiteEUI(kBtu/sf)']

#### Colonne SiteEnergyUseWN(kBtu)

In [ ]:
var = "SiteEnergyUseWN(kBtu)"

mask_zero = df[var].eq(0)

if not mask_zero.any():
    print("   ✅ Aucun zéro détecté")
else:
    recalculables = mask_zero & df["SiteEUIWN(kBtu/sf)"].gt(0) & df["PropertyGFATotal"].gt(0)

    df.loc[recalculables, var] = (
        df.loc[recalculables, "SiteEUIWN(kBtu/sf)"] *
        df.loc[recalculables, "PropertyGFATotal"]
    )

    print(f"   Zéros détectés: {mask_zero.sum()}")
    print(f"   ✅ {recalculables.sum()} valeur(s) recalculée(s)")

  

In [ ]:
display_columns_with_zero(df)

In [ ]:
display(df.describe())

### 10.3 Valeurs abérrantes

In [ ]:
critere = df['NumberofBuildings'] >= 9
col = [
    'PropertyName',
	'BuildingType',
	'PrimaryPropertyType',
	'YearBuilt',
	'NumberofBuildings',
	'NumberofFloors',
    'PropertyGFATotal',
    'PropertyGFAParking',
    'PropertyGFABuilding(s)'
]
print(len(df[critere]))
display(df[critere][col].sort_values(by='NumberofBuildings'))

**Observations**

Le nombre de batiment semble cohérent même pour *University of Washington - Seattle Campus* qui à 111 batiments, une simple recherche sur internet nous permet de voir que ce chiffre est correct.


In [ ]:
critere = df['NumberofFloors'] >= 50
col = [
    'PropertyName',
	'Adress'
	'BuildingType',
	'PrimaryPropertyType',
	'YearBuilt',
	'NumberofBuildings',
	'NumberofFloors',
    'PropertyGFATotal',
    'PropertyGFAParking',
    'PropertyGFABuilding(s)'
]
print(len(df[critere]))
display(df[critere].sort_values(by='NumberofBuildings'))

**Observations**

Seul le bâtiment *Seattle Chinese Baptist Church* semble présenter une valeur aberrante pour le nombre d’étages : 99 étages, ce qui est excessivement élevé pour une église.  
De plus, la consultation de Wikipédia, qui recense les bâtiments les plus hauts de Seattle, permet de valider les valeurs des autres bâtiments. Cette source confirme que l’église constitue une anomalie et nécessite une correction.  
https://fr.wikipedia.org/wiki/Liste_des_plus_hautes_constructions_de_Seattle

**Décision**

Nous allons corriger cette valeur en utilisant la médiane de *NumberOfFloors* pour la catégorie **Worship Facilities**.

In [ ]:
# Calculer la médiane des NumberofFloors pour les Worship Facilities
median_worship = df[df['PrimaryPropertyType'] == 'Worship Facility']['NumberofFloors'].median()
print(f"Médiane des étages pour les lieux de culte : {median_worship:.0f}")

# Corriger la valeur aberrante
df.loc[df['NumberofFloors'] == 99, 'NumberofFloors'] = median_worship

# Vérifier la correction
print(f"\n✅ Correction effectuée !")
print(f"Nouvelle valeur pour Seattle Chinese Baptist Church : {median_worship:.0f} étages")

## 11 Résumé après nettoyage

In [ ]:

print("="*80)
print("📊 ANALYSE DES VALEURS MANQUANTES")
print("="*80)

print(f"\n🔧 Shape du DataFrame avant traitement des valeurs manquantes : {df.shape}")

missing_stats = analyze_missing_values(df, True)

print(f"\n Distribution finale des valeurs numériques après nettoyage :")
display(df.describe())

print(f"\n Informations sur le DataFrame final :")
display(df.info())

## 12. Les 3 types de relations entre variables

### 12.1. Quantitative vs Quantitative

**Objectif :** Mesurer la relation linéaire entre deux variables numériques.

**Graphiques :**
- Scatter plot (nuage de points)
- Matrice de corrélation (heatmap)

**Exemples :**
- Surface totale → Consommation d'énergie et Émissions CO2
- Année de construction → Consommation d'énergie et Émissions CO2

#### 12.1.1 Matrice de corrélation générale

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

In [ ]:
quanti_vars = [
    'PropertyGFATotal',
    'PropertyGFABuilding(s)',
    'YearBuilt',
    'NumberofFloors',
    TARGET_CO2,
    TARGET_ENERGY
]


df_quanti = df[quanti_vars]

# Calcul de la matrice de corrélation
correlation_matrix = df_quanti.corr()

# Visualisation : Heatmap de corrélation
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matrice de corrélation - Variables quantitatives', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

#### 12.1.2 Surface totale vs ( Consommation d'énergie et Emission de CO2 )

Afin d’analyser l’impact de la surface totale des bâtiments sur leurs performances énergétiques et environnementales, nous comparons la surface totale à la consommation d’énergie ainsi qu’aux émissions de CO₂ à l’aide de nuages de points. Ces graphiques permettent d’observer la relation entre la taille des bâtiments et ces deux indicateurs, d’identifier d’éventuelles corrélations et de repérer des bâtiments atypiques

In [ ]:
def create_scatter(
    df: pd.DataFrame,
    var1: str,
    var2: str,
    title: str,
    xlabel: str,
    ylabel: str,
    ax: plt.Axes,
    add_trendline: bool = False
):
    if add_trendline:
        sns.regplot(
            data=df,
            x=var1,
            y=var2,
            ax=ax,
            scatter_kws={
                "alpha": 0.5,
                "edgecolor": "k"
            },
            line_kws={
                "color": "red",
                "linewidth": 2
            },
            ci=None
        )
    else:
        sns.scatterplot(
            data=df,
            x=var1,
            y=var2,
            ax=ax,
            alpha=0.5,
            edgecolor="k"
        )

    ax.set_xlabel(xlabel, fontsize=11)
    ax.set_ylabel(ylabel, fontsize=11)
    ax.set_title(title, fontsize=13, fontweight="bold")
    ax.grid(True, alpha=0.3)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

create_scatter(
    df,
    "PropertyGFATotal",
    TARGET_ENERGY,
    "Relation entre la surface totale et la consommation d'énergie",
    "Surface totale du bâtiment (PropertyGFATotal)",
    "Consommation d'énergie (kBtu)",
    axes[0]
)

create_scatter(
    df,
    "PropertyGFATotal",
    TARGET_CO2,
    "Relation entre la surface totale et les émissions de CO₂",
    "Surface totale du bâtiment (PropertyGFATotal)",
    "Émissions CO2 (TotalGHGEmissions)",
    axes[1]
)

plt.tight_layout()
plt.show()



##### Analyse synthétique des relations surface – énergie et CO₂

Les deux graphiques montrent une **relation positive globale** entre la surface totale des bâtiments et :
- la **consommation d’énergie** ;
- les **émissions de CO₂**.

En général, plus un bâtiment est grand, plus il consomme d’énergie et plus il émet de CO₂.  
Cependant, pour une **même surface**, on observe une **forte variabilité** des consommations et des émissions, ce qui indique que la surface seule n’explique pas tout.

Cette dispersion s’explique notamment par :
- le **type et l’usage** des bâtiments ;
- leur **performance énergétique** ;
- le **type d’énergie utilisée**, qui influence directement les émissions de CO₂.

Quelques bâtiments de très grande surface présentent des valeurs extrêmes, ce qui souligne l’intérêt de :
- normaliser les indicateurs (énergie/m², CO₂/m²) ;
- analyser séparément certains bâtiments atypiques.

**Conclusion :**  
La surface est un facteur explicatif important, mais insuffisant à elle seule. Une analyse plus fine nécessite des indicateurs normalisés et une segmentation par usage ou type de bâtiment.


#### 12.1.3 Année de construction vs Consommation d'énergie

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

create_scatter(
    df,
    "YearBuilt",
    TARGET_ENERGY,
    "Consommation d'énergie selon l'année de construction",
    "Année de construction",
    "Consommation d'énergie (kBtu)",
    axes[0]
)

create_scatter(
    df,
    "YearBuilt",
    TARGET_CO2,
    "Émissions de CO2 selon l'année de construction",
    "Année de construction",
    "Émissions CO2 (TotalGHGEmissions)",
    axes[1]
)

plt.tight_layout()
plt.show()

### Analyse synthétique – année de construction, énergie et CO₂

Aucune tendance linéaire forte n’apparaît clairement entre l’année de construction et les niveaux de consommation ou d’émissions. Des bâtiments anciens comme récents peuvent présenter des consommations et des émissions élevées.

On observe toutefois que :
- la majorité des bâtiments, quelle que soit leur année de construction, a des **consommations et émissions relativement faibles** ;
- quelques **valeurs extrêmes** sont présentes sur l’ensemble de la période, ce qui suggère que l’année de construction seule n’est pas un facteur explicatif suffisant.

Ces résultats indiquent que :
- l’**usage du bâtiment**, sa **taille**, et ses **rénovations énergétiques** ont probablement un impact plus important que l’année de construction initiale ;
- les bâtiments récents ne sont pas systématiquement les plus performants, et certains bâtiments anciens peuvent être efficaces s’ils ont été rénovés.

**Conclusion :**  
L’année de construction, prise isolément, explique peu la consommation d’énergie et les émissions de CO₂. Une analyse plus pertinente nécessiterait d’intégrer des informations sur les rénovations, les usages et des indicateurs normalisés (énergie/m², CO₂/m²).


### 12.2. Quantitative vs Qualitative

**Objectif :** Comparer une variable numérique selon différentes catégories.

**Mesure :** ANOVA, Rapport de corrélation η² (entre 0 et 1)

**Graphiques :**
- Boxplot
- Violin plot
- Bar plot (moyennes)

**Exemples :**
- Consommation d'énergie et Émissions CO2 selon le type de propriété
- Consommation d'énergie et Émissions CO2 selon le quartier
- Consommation d'énergie et Émissions CO2 selon le type de bâtiment

#### 12.2.1 Consommation d'énergie selon le type de propriété

In [ ]:
def create_boxplot(
    df: pd.DataFrame,
    y_col: str,
    ax: plt.Axes,
    x_col: str = None,
    title: str = "",
    xlabel: str = "",
    ylabel: str = "",
    rotate_x: int = 0,
    log_scale: bool = False
):
    sns.boxplot(
        data=df,
        x=x_col,
        y=y_col,
        ax=ax
    )

    if log_scale:
        ax.set_yscale("log")
        title = title + "(log scale)"
        ylabel = ylabel + "(log scale)"

    if rotate_x:
        ax.tick_params(axis='x', rotation=rotate_x)


    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)


In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
create_boxplot(
	df,
	TARGET_ENERGY,
	ax,
	'PrimaryPropertyType', 
	'Consommation d\'énergie selon le type de propriété',
	'Type de propriété', 
	'Consommation d\'énergie (kBtu)'
	)
plt.tight_layout()
plt.show()

##### Analyse synthétique – consommation d’énergie selon le type de propriété

On observe des **différences marquées entre les catégories** :
- Les **hôpitaux** présentent de loin les consommations d’énergie les plus élevées, avec une médiane importante et une forte dispersion. Cela s’explique par une activité continue, des équipements lourds et des exigences sanitaires élevées.
- Les **universités**, les **laboratoires**, les **supermarchés** et les **grands bureaux** affichent également des niveaux de consommation plus élevés que la moyenne.
- Les **écoles K-12**, les **bureaux**, les **restaurants**, les **entrepôts** et les **logements collectifs de faible hauteur** présentent des consommations globalement plus faibles.

La présence de nombreux **outliers** dans plusieurs catégories indique une **forte hétérogénéité interne**, liée notamment à la taille des bâtiments, à leur usage réel et à leur intensité d’exploitation.

**Conclusion :**  
Le type de propriété est un facteur déterminant de la consommation d’énergie. Toutefois, les écarts observés au sein d’une même catégorie montrent que des indicateurs normalisés (énergie/m²) et une segmentation plus fine restent nécessaires pour comparer efficacement la performance énergétique des bâtiments.


#### 12.2.2 Émissions CO2 selon le type de propriété

In [ ]:
# Visualisation : Boxplot
fig, ax = plt.subplots(figsize=(14, 6))
create_boxplot(df, TARGET_CO2, ax, 'PrimaryPropertyType', 'Émissions CO2 selon le type de propriété', 'Type de propriété', 'Émissions CO2 (TotalGHGEmissions)')
plt.tight_layout()
plt.show()

##### Analyse synthétique – émissions de CO₂ selon le type de propriété

Des écarts importants apparaissent entre les catégories :
- Les **hôpitaux** sont de loin les plus émetteurs, avec des valeurs médianes élevées et une dispersion très marquée. Cela reflète une consommation énergétique continue et souvent basée sur des sources fortement carbonées.
- Les **universités**, les **laboratoires**, les **supermarchés / épiceries** et les **grands bureaux** présentent également des niveaux d’émissions plus élevés que la moyenne.
- Les **bureaux standards**, **écoles K-12**, **entrepôts**, **restaurants** et **bâtiments résidentiels de faible hauteur** affichent des émissions globalement plus faibles.

La présence de nombreux **outliers** dans plusieurs catégories souligne une **forte hétérogénéité** au sein d’un même type de propriété, liée à la taille, à l’intensité d’usage et au mix énergétique.

**Conclusion :**  
Le type de propriété est un facteur clé des émissions de CO₂, mais il ne suffit pas à lui seul pour comparer la performance environnementale. L’utilisation d’indicateurs normalisés (CO₂/m² ou CO₂ par usage) et une analyse par type d’énergie permettraient une interprétation plus robuste.


#### 12.2.3 Consommation d’énergie et émissions de CO₂ selon le quartier

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
create_boxplot(df, TARGET_ENERGY,ax , 'Neighborhood', 'Consommation d\'énergie selon le Quartier','Quartier', 'Consommation d\'énergie (kBtu)')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
create_boxplot(df, TARGET_CO2, ax,  'Neighborhood', 'Émissions CO2 selon le quartier','Quartier', 'Émissions CO2 (TotalGHGEmissions)')
plt.tight_layout()
plt.show()

##### Analyse synthétique – consommation d’énergie et émissions de CO₂ selon le quartier

Globalement, on n’observe pas de différences extrêmement marquées entre les quartiers en termes de valeurs médianes : la majorité des bâtiments, quel que soit le quartier, présente des **consommations et émissions relativement faibles**.

Cependant, certains quartiers se distinguent par la présence de **valeurs extrêmes (outliers)** :
- **Downtown**, **East**, **Northeast** et **Greater Duwamish** concentrent plusieurs bâtiments avec des consommations d’énergie et des émissions de CO₂ très élevées.
- Ces quartiers sont probablement caractérisés par une plus forte densité de bâtiments de grande taille ou à usage intensif (hôpitaux, universités, bâtiments industriels ou tertiaires).

Les quartiers plus résidentiels ou moins denses (par exemple **North**, **Ballard**, **Magnolia / Queen Anne**, **Delridge**) présentent en général des niveaux plus faibles et moins dispersés.

**Conclusion :**  
Le quartier influence indirectement la consommation d’énergie et les émissions de CO₂, principalement via le **type de bâtiments** et leur **usage**. À lui seul, le quartier n’est pas un facteur explicatif suffisant. Une analyse combinant quartier, type de propriété et indicateurs normalisés (énergie/m², CO₂/m²) permettrait une interprétation plus robuste.


### 3. Qualitative vs Qualitative

**Objectif :** Analyser la dépendance entre deux variables catégorielles.

**Mesure :** Test du Chi², V de Cramér (entre 0 et 1)

**Graphiques :**
- Heatmap de contingence
- Stacked bar plot
- Tableau de contingence

**Exemples :**
- Type de bâtiment vs Quartier
- Type de bâtiment vs Type d'usage

In [ ]:
def create_barplot_contingency(
    df: pd.DataFrame,
    x_col: str,
    hue_col: str,
    title: str,
    xlabel: str,
    ylabel: str,
    legend_title: str,
    top_n_x: int | None = None,
    top_n_y: int | None = None
):
    
    df_copy = df.copy()
    # Optionnel : limiter la cardinalité
    if top_n_x is not None:
        top_x = df_copy[x_col].value_counts().head(top_n_x).index
        df_copy = df_copy[df_copy[x_col].isin(top_x)]

    if top_n_y is not None:
        top_y = df_copy[hue_col].value_counts().head(top_n_y).index
        df_copy = df_copy[df_copy[hue_col].isin(top_y)]

    contingency = (
        df_copy
        .groupby([x_col, hue_col])
        .size()
        .reset_index(name='count')
    )

    plt.figure(figsize=(12, 6))
    sns.barplot(
        data=contingency,
        x=x_col,
        y='count',
        hue=hue_col
    )

    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title=legend_title, )
    plt.tight_layout()
    plt.show()


def create_heatmap_contingency(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    title: str,
    xlabel: str,
    ylabel: str,
    top_n_x: int | None = None,
    top_n_y: int | None = None,
    normalize: bool = False,
    annot: bool = False,
    fmt: str = 'd'
):
    data = df.copy()

    if top_n_x is not None:
        top_x = data[x_col].value_counts().head(top_n_x).index
        data = data[data[x_col].isin(top_x)]

    if top_n_y is not None:
        top_y = data[y_col].value_counts().head(top_n_y).index
        data = data[data[y_col].isin(top_y)]

    contingency = pd.crosstab(
        data[y_col],
        data[x_col],
        normalize='index' if normalize else False
    )

    if annot and fmt is None:
        fmt = '.2f' if normalize else 'd'

    plt.figure(figsize=(12, 6))
    sns.heatmap(
        contingency if not normalize else contingency * 100,
        cmap='Blues',
        annot=annot,
        fmt=fmt,
        linewidths=0.5
    )

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


In [ ]:
display(df.head())
create_barplot_contingency(
    df,
    'Neighborhood',
    'BuildingType',
    'Répartition des types de bâtiments par quartier',
    'Quartier',
    'Nombre de bâtiments',
    'Type de bâtiment'
)
create_heatmap_contingency(
    df,
    x_col='Neighborhood',
    y_col='BuildingType',
    title="Heatmap – Types de bâtiments par quartier",
    xlabel="Quartier",
    ylabel="Type de bâtiment",
    normalize=False,  # True si tu veux des proportions
    annot=True
)


#### Analyse synthétique – répartition des types de bâtiments par quartier

Les résultats mettent en évidence une **forte domination des bâtiments non résidentiels** dans l’ensemble des quartiers. Cette domination est particulièrement marquée dans :
- **Downtown** et **Greater Duwamish**, qui concentrent de loin le plus grand nombre de bâtiments non résidentiels ;
- **Lake Union**, **Magnolia / Queen Anne**, **East** et **Northeast**, qui présentent également des volumes significatifs.

Les autres catégories de bâtiments sont **largement minoritaires** :
- Les **Nonresidential COS** et **SPS-District K-12** sont présents dans presque tous les quartiers, mais en quantités limitées.
- Les **campus** sont très peu nombreux et localisés dans quelques quartiers spécifiques.
- Les **Nonresidential WA** sont quasi inexistants dans l’échantillon.

La heatmap confirme visuellement ces tendances en montrant une **forte concentration** de bâtiments non résidentiels dans certains quartiers, tandis que les autres types restent dispersés et peu représentés.

**Conclusion :**  
La structure du parc immobilier varie fortement selon les quartiers, avec une concentration des bâtiments non résidentiels dans les zones à forte activité économique (notamment Downtown et Greater Duwamish). Cette répartition explique en partie les différences observées précédemment en matière de consommation d’énergie et d’émissions de CO₂, soulignant l’importance de croiser **quartier**, **type de bâtiment** et **usage** pour une analyse énergétique pertinente.


In [ ]:
create_barplot_contingency(
    df,
    'Neighborhood',
    'PrimaryPropertyType',
    'Répartition du top 5 types de propriétés par quartier',
    'Quartier',
    'Nombre de bâtiments',
    'Type de propriété',
    top_n_y=5
)
create_heatmap_contingency(
    df,
    x_col='Neighborhood',
    y_col='PrimaryPropertyType',
    title="Heatmap – Top 10 des propriétés par quartier",
    xlabel="Quartier",
    ylabel="Type de propriétés",
    top_n_y=10,      # fortement recommandé
    normalize=False,  # True si tu veux des proportions
    annot=True
)

#### Analyse synthétique – répartition des types de propriétés par quartier (Top 5 et Top 10)

Ces deux visualisations présentent la **distribution des principaux types de propriétés selon les quartiers**, à travers :
- un graphique en barres pour le **Top 5 des types de propriétés** ;
- une heatmap pour le **Top 10 des types de propriétés**.

##### Tendances générales
- Certains quartiers concentrent très fortement des **types de propriétés spécifiques**, révélant une spécialisation fonctionnelle claire.
- **Downtown** se distingue par une forte concentration de **Large Offices**, **Small and Mid-Sized Offices**, ainsi que de **Mixed Use Properties**, ce qui confirme son rôle de pôle tertiaire majeur.
- **Greater Duwamish** est largement dominé par les **Warehouses** et les **Distribution Centers**, reflétant une vocation principalement industrielle et logistique.

##### Répartition par type de propriété
- Les **bureaux (Large et Small/Mid-Sized)** sont surtout présents à **Downtown**, **Lake Union** et **Magnolia / Queen Anne**.
- Les **entrepôts (Warehouses)** sont massivement concentrés à **Greater Duwamish**, avec une présence marginale dans les autres quartiers.
- Les **K-12 Schools** sont réparties de manière relativement homogène entre les quartiers, indiquant une implantation plus résidentielle et territoriale.
- Les catégories **Retail Store**, **Hotel** et **Worship Facility** apparaissent de façon plus diffuse, sans concentration extrême, mais avec des pics locaux selon les quartiers.

##### Lecture croisée des deux graphiques
- Le graphique Top 5 met en évidence les **dominances locales**, tandis que la heatmap Top 10 permet d’identifier les **types secondaires mais structurants** dans chaque quartier.
- Ensemble, ils montrent que la **composition du parc immobilier varie fortement d’un quartier à l’autre**, bien plus que le simple nombre total de bâtiments.

##### Conclusion
La répartition des types de propriétés par quartier révèle une **forte spécialisation spatiale** (tertiaire, industrielle, mixte ou résidentielle). Cette structure explique en grande partie les différences observées précédemment en matière de **consommation d’énergie** et d’**émissions de CO₂**, et souligne l’importance d’intégrer le **type de propriété** dans toute analyse énergétique ou environnementale à l’échelle urbaine.


## 13. Récapitulatif de l'analyse exploratoire 
   
##### Dataset initial
- Lignes : 3376  
- Colonnes : 46
- Pourcentage de cellule vide : 12.85%
   
##### Dataset après nettoyage exploratoire
- Lignes : 1630 (après filtrage type de bâtiment + outliers + compliance)  
- Colonnes : 39 
- Pourcentage de cellule vide : 0 % 
   
##### Décisions prises
1. Variables cibles : `TotalGHGEmissions` et `SiteEnergyUse(kBtu)`  
2. Colonnes supprimées : `Outlier`, `Comments`, `YearsENERGYSTARCertified`, `SecondLargestPropertyUseType`, `SecondLargestPropertyUseTypeGFA` ,`ThirdLargestPropertyUseType`,`ThirdLargestPropertyUseTypeGFA`, `ENERGYSTARScore`
3. Lignes supprimées : outliers, bâtiments non conformes, données manquantes  
4. Type de bâtiments conservés : bâtiments non résidentiels uniquement  
   
##### Insights clés pour la modélisation

- La **surface totale du bâtiment** est fortement corrélée aux deux variables cibles, mais ne suffit pas à expliquer seule la variabilité observée  
  → nécessité d’intégrer des variables complémentaires et/ou des indicateurs normalisés (énergie/m², CO₂/m²).

- Le **type de propriété** est l’un des facteurs les plus discriminants pour l’énergie et le CO₂  
  → variable catégorielle clé à encoder soigneusement (one-hot ou target encoding selon le modèle).

- L’**année de construction** présente un pouvoir explicatif limité prise isolément  
  → variable à conserver, mais avec un poids attendu modéré dans les modèles.

- Une **forte hétérogénéité intra-catégorie** (notamment pour les hôpitaux, universités et grands bureaux) est observée  
  → intérêt potentiel de modèles non linéaires capables de capturer des interactions complexes.

- La **localisation (quartier)** influence indirectement les cibles via la structure du parc immobilier  
  → variable utile en interaction avec le type de propriété plutôt qu’en effet direct.

- La présence de **valeurs extrêmes** justifie :
  - le filtrage des outliers en amont ;
  - l’utilisation de métriques robustes et/ou de transformations (log, scaling).

- La variabilité importante des consommations et émissions suggère que des **modèles multivariés** seront plus adaptés que des approches univariées ou purement linéaires.

Ces éléments orientent vers une approche de modélisation intégrant à la fois des **variables structurelles**, **fonctionnelles** et **spatiales**, avec une attention particulière portée à la normalisation et aux interactions entre variables.


## 14. Feature Engineering

### 14.1. Suppression des colonnes peu pertinentes

Nettoyer le dataset en supprimant les colonnes qui n'apportent aucune information utile pour l'analyse et la modélisation.

In [ ]:
low_predictive_value_columns = [
    'DataYear', 
    'City', 
    'State',
	'OSEBuildingID', 
    'PropertyName', 
    'Address', 
    'TaxParcelIdentificationNumber',
	'DefaultData', 
    'ComplianceStatus',
]
display_columns_info(df, low_predictive_value_columns)

##### Colonnes à valeur unique (3)
Ces colonnes contiennent la même valeur pour tous les bâtiments, elles n'apportent donc aucune variabilité ni pouvoir discriminant.

| Colonne | Valeur unique | Raison |
|---------|---------------|--------|
| `DataYear` | 2016 | Toutes les observations datent de 2016 |
| `City` | Seattle | Tous les bâtiments sont situés à Seattle |
| `State` | WA | Tous les bâtiments sont dans l'État de Washington |

##### Identifiants uniques (4)
Ces colonnes identifient de manière unique chaque bâtiment mais ne permettent pas de généraliser à de nouveaux bâtiments.

| Colonne | Taux d'unicité | Raison |
|---------|----------------|--------|
| `OSEBuildingID` | 100% | Identifiant système unique par bâtiment |
| `PropertyName` | 99.8% | Nom propre du bâtiment (ex: "Mayflower Park Hotel") |
| `Address` | 98.7% | Adresse complète spécifique à chaque bâtiment |
| `TaxParcelIdentificationNumber` | 95% | Numéro de parcelle fiscale unique |

##### Métadonnées (2)
Ces colonnes concernent la qualité et le statut des données, pas les caractéristiques des bâtiments.

| Colonne | Type | Raison |
|---------|------|--------|
| `DefaultData` | Booléen | Indique si les données sont estimées ou réelles |
| `ComplianceStatus` | Texte | Statut de conformité réglementaire ("Compliant" ou "Error") |

In [ ]:
df = remove_columns(df, low_predictive_value_columns)

### 14.2  Analyse des distributions et gestion des outliers

In [ ]:
def create_histogram(
    df: pd.DataFrame,
    column: str,
    title: str,
    xlabel: str,
    ylabel: str,
    ax: plt.Axes,
    log_scale: bool = False
):
    sns.histplot(
        data=df,
        x=column,
        bins=50,
        kde=False,
        ax=ax,
        edgecolor='black'
    )

    if log_scale:
        ax.set_yscale("log")
        title = title + "(log scale)"
        ylabel = ylabel + "(log scale)"

    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

create_histogram(
    df,
    TARGET_ENERGY,
    'Distribution de TotalGHGEmissions',
    'TotalGHGEmissions',
    'Fréquence',
    axes[0],
    log_scale=True

)


create_histogram(
    df,
    TARGET_CO2,
    'Distribution de SiteEnergyUse(kBtu)',
    'SiteEnergyUse(kBtu)',
    'Fréquence',
    axes[1],
    log_scale=True
)

plt.tight_layout()
plt.show()


# Boxplots pour identifier les outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

create_boxplot(
    df,
    y_col=TARGET_ENERGY,
    title='Boxplot SiteEnergyUse(kBtu)',
    xlabel='SiteEnergyUse(kBtu)',
    ylabel='SiteEnergyUse(kBtu)',
    ax=axes[1],
    log_scale=True
)

create_boxplot(
    df,
    y_col=TARGET_CO2,
    title='Boxplot TotalGHGEmissions',
    xlabel='TotalGHGEmissions',
    ylabel='TotalGHGEmissions',
    ax=axes[0],
    log_scale=True
)

plt.tight_layout()
plt.show()

#### Analyse des graphiques

- **TotalGHGEmissions** :  
  La distribution est très étalée vers la droite. On observe que la plupart des valeurs sont faibles, mais quelques bâtiments ont des émissions très élevées. Ces valeurs extrêmes (outliers) influencent fortement la distribution. Le boxplot montre une médiane plutôt basse et une grande dispersion.

- **SiteEnergyUse (kBtu)** :  
  La consommation d’énergie est elle aussi majoritairement faible à modérée pour la plupart des sites. Cependant, certains bâtiments consomment énormément d’énergie, ce qui crée des outliers visibles sur le boxplot. La distribution n’est donc pas symétrique.

**Conclusion** :  
Les deux variables présentent des distributions asymétriques avec beaucoup de valeurs extrêmes. Pour aller plus loin dans l’analyse, il pourrait être utile de traiter les outliers afin d’obtenir des résultats plus stables.


In [ ]:
cibles = ['TotalGHGEmissions', 'SiteEnergyUse(kBtu)']

for cible in cibles:
    print(f"\n--- {cible} ---")
    
    # Calculer Q1, Q3 et IQR
    Q1 = df[cible].quantile(0.25)
    Q3 = df[cible].quantile(0.75)
    IQR = Q3 - Q1
    
    # Définir les bornes
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identifier les outliers
    outliers = df[(df[cible] < lower_bound) | (df[cible] > lower_bound)][cible]
    
    print(f"Q1 (25%) : {Q1:,.2f}")
    print(f"Q3 (75%) : {Q3:,.2f}")
    print(f"IQR : {IQR:,.2f}")
    print(f"Borne inférieure : {lower_bound:,.2f}")
    print(f"Borne supérieure : {upper_bound:,.2f}")
    print(f"\nNombre d'outliers : {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    
    if len(outliers) > 0:
        print(f"Valeur minimale des outliers : {outliers.min():,.2f}")
        print(f"Valeur maximale des outliers : {outliers.max():,.2f}")

    
    
    # Créer une colonne indicatrice
    df[f'{cible}_outlier_IQR'] = (df[cible] < lower_bound) | (df[cible] > upper_bound)

    # ========================================
    # VISUALISATION COMPARATIVE
    # ========================================
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Graphique 3 : IQR
    axes[0].hist(df[cible].dropna(), bins=50, alpha=0.6, 
                    label='Données normales', edgecolor='black', color='lightblue')
    axes[0].hist(outliers, bins=30, alpha=0.9, 
                    label=f'Outliers ({len(outliers)})', color='green', edgecolor='black')
    axes[0].axvline(upper_bound, color='darkgreen', linestyle='--', linewidth=2, 
                    label=f'Seuil IQR = {upper_bound:,.0f}')
    axes[0].set_xlabel('SiteEnergyUseWN (kBtu)')
    axes[0].set_ylabel('Fréquence')
    axes[0].set_title(f'IQR (facteur 1.5) ({len(outliers)} outliers - {(len(outliers)/len(df)*100):.1f}%)')
    axes[0].legend()
    axes[0].set_yscale('log')
    axes[0].grid(alpha=0.3)

    # Graphique 4 : Boxplot comparatif

    data_iqr = df[cible].copy()
    data_iqr.loc[outliers.index] = np.nan

    box_data = [
        df[cible].dropna(),
        data_iqr.dropna()
    ]

    bp = axes[1].boxplot(box_data, tick_labels=['Originales', 'Sans IQR'],
                            patch_artist=True)
    for patch, color in zip(bp['boxes'], ['lightgray', 'green']):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    axes[1].set_ylabel(cible)
    axes[1].set_title('Comparaison des distributions après suppression des outliers')
    axes[1].set_yscale('log')
    axes[1].grid(alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

#### Outliers = Bâtiments exceptionnels ?
Les outliers détectés (10-11% des données) correspondent probablement à :

- Grands campus universitaires (ex: University of Washington)
- Hôpitaux majeurs avec équipements énergétivores
- Grands centres commerciaux ou complexes de bureaux
- Bâtiments industriels à forte consommation


#### Supprimer les outliers

Impact sur le modèle : Les outliers peuvent biaiser les coefficients de régression
Distribution normale : Certains algorithmes performent mieux sur des distributions normales
Généralisation : Le modèle pourrait mieux prédire les bâtiments "typiques"

In [ ]:
print("\n" + "="*80)
print("SUPPRESSION DES OUTLIERS")
print("="*80)
# Supprimer si outlier pour AU MOINS UNE des deux cibles (RECOMMANDÉ)
outliers_mask = df[TARGET_ENERGY + '_outlier_IQR'] | df[TARGET_CO2 + '_outlier_IQR']


print(f"\nNombre de bâtiments à supprimer : {outliers_mask.sum()} ({outliers_mask.sum()/len(df)*100:.1f}%)")

# Créer le DataFrame nettoyé
df_clean = df[~outliers_mask].copy()

# Supprimer les colonnes indicatrices (optionnel)
df_clean = df_clean.drop(columns=[TARGET_ENERGY + '_outlier_IQR', TARGET_CO2 + '_outlier_IQR'])

print(f"\nDimensions APRÈS suppression : {df_clean.shape}")
print(f"Nombre de bâtiments conservés : {len(df_clean)}")
print(f"Nombre de bâtiments supprimés : {len(df) - len(df_clean)}")

df = df_clean

### 14.3. Suppression des features redondantes

Identifier et supprimer les features redondantes en utilisant une matrice de corrélation de Pearson pour éviter la multicolinéarité et simplifier le modèle.

**Principe**
La corrélation de Pearson mesure la relation linéaire entre deux variables numériques :
- **r = 1** : Corrélation positive parfaite
- **r = 0** : Aucune corrélation
- **r = -1** : Corrélation négative parfaite

In [ ]:
# 1. Calculer la matrice de corrélation
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = df_clean[numeric_cols].corr()

# 2. Visualiser avec Seaborn
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

sns.heatmap(corr_matrix, 
            annot=True, 
            fmt='.2f', 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=0.5,
            mask=mask,
            vmin=-1, vmax=1)

plt.title('Matrice de corrélation de Pearson', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

#### Features redondantes identifiées (corrélation > 0.95)

| Paire de variables | Corrélation | Raison | Action |
|-------------------|-------------|--------|--------|
| `Electricity(kWh)` ↔ `Electricity(kBtu)` | **1.000** | Même variable, unités différentes | ✅ Supprimer `Electricity(kWh)` |
| `NaturalGas(therms)` ↔ `NaturalGas(kBtu)` | **1.000** | Même variable, unités différentes | ✅ Supprimer `NaturalGas(therms)` |
| `SourceEUI(kBtu/sf)` ↔ `SourceEUIWN(kBtu/sf)` | **1.000** | Version normalisée météo | ✅ Supprimer `SourceEUIWN(kBtu/sf)` |
| `SiteEnergyUse(kBtu)` ↔ `SiteEnergyUseWN(kBtu)` | **1.000** | Version normalisée météo | ✅ Supprimer `SiteEnergyUseWN(kBtu)` |
| `SiteEUI(kBtu/sf)` ↔ `SiteEUIWN(kBtu/sf)` | **1.000** | Version normalisée météo | ✅ Supprimer `SiteEUIWN(kBtu/sf)` |


**Pourquoi garder kBtu et supprimer kWh/therms ?**
- kBtu (British Thermal Unit) est l'unité standard pour l'énergie dans le bâtiment
- Permet de comparer directement électricité, gaz, vapeur sur la même échelle
- 1 kWh = 3.412 kBtu (conversion linéaire parfaite)
- 1 therm = 100 kBtu (conversion linéaire parfaite)

**Pourquoi supprimer les versions normalisées ?**
- Les versions WN ajustent la consommation selon les variations météo
- Corrélation quasi-parfaite (>0.997) avec les versions non normalisées
- Pour un premier modèle, les versions non normalisées suffisent
- Les versions WN pourraient être utiles dans un modèle avancé

#### Corrélations fortes mais NON redondantes (0.7 < r < 0.95)

Ces paires ont des corrélations fortes mais représentent des informations différentes :

| Paire | Corrélation | Pourquoi les garder toutes les deux ? |
|-------|-------------|----------------------------------------|
| `SiteEUI(kBtu/sf)` ↔ `SourceEUI(kBtu/sf)` | 0.94 | Site vs Source (énergie consommée vs énergie primaire) |
| `PropertyGFATotal` ↔ `PropertyGFABuilding(s)` | 0.93 | Total vs Bâtiments uniquement (différence = parking) |
| `SiteEnergyUse(kBtu)` ↔ `Electricity(kBtu)` | 0.92 | Total vs Composante principale |
| `NaturalGas(kBtu)` ↔ `TotalGHGEmissions` | 0.91 | Consommation vs Émissions |

**Décision** : Ces features sont **conservées** car elles apportent une information complémentaire.

In [ ]:
features_redondantes = [
    'Electricity(kWh)',
    'NaturalGas(therms)',
    'SiteEUIWN(kBtu/sf)',
    'SourceEUIWN(kBtu/sf)',
    'SiteEnergyUseWN(kBtu)'
]

df = remove_columns(df, features_redondantes)

### 14.4. Visualisation des relations features ↔ targets

### 1. Scatterplots : Features numériques vs Cibles
**Fichier** : `scatterplots_features_vs_targets.png`

Visualisation de 6 features structurelles importantes vs les 2 cibles :
- `PropertyGFATotal` (Surface totale)
- `PropertyGFABuilding(s)` (Surface bâtiments)
- `LargestPropertyUseTypeGFA` (Surface usage principal)
- `NumberofFloors` (Nombre d'étages)
- `YearBuilt` (Année de construction)
- `NumberofBuildings` (Nombre de bâtiments)

In [ ]:
# Sélectionner les features structurelles importantes
features_numeriques = [
    'PropertyGFATotal',
    'PropertyGFABuilding(s)',
    'LargestPropertyUseTypeGFA',
    'NumberofFloors',
    'YearBuilt',
    'NumberofBuildings'
]

# Créer une grille de scatterplots
fig, axes = plt.subplots(len(features_numeriques), 2, figsize=(16, 20))
fig.suptitle('Relation entre features structurelles et variables cibles', 
            fontsize=16, fontweight='bold')

for i, feature in enumerate(features_numeriques):
    for j, cible in enumerate(cibles):
        # Scatterplot
        axes[i, j].scatter(df_clean[feature], df_clean[cible], 
                          alpha=0.5, s=20, color='steelblue')
        axes[i, j].set_xlabel(feature, fontsize=10)
        axes[i, j].set_ylabel(cible, fontsize=10)
        
        # Calculer et afficher la corrélation
        corr = df_clean[feature].corr(df_clean[cible])
        axes[i, j].text(0.05, 0.95, f'r = {corr:.3f}', 
                       transform=axes[i, j].transAxes, 
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                       verticalalignment='top', fontsize=9)
        axes[i, j].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('scatterplots_features_vs_targets.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Scatterplots créés")

**Observations clés** :

- **PropertyGFATotal**  
  Plus la surface totale du bâtiment (batiment + parking) est grande, plus les émissions de CO₂ et la consommation d’énergie ont tendance à être élevées.

- **PropertyGFABuilding(s)**  
  Surface des bâtiments est fortement liée aux émissions et à l’énergie consommée, ce qui en fait une variable très importante.

- **LargestPropertyUseTypeGFA**  
  Quand la surface du principal usage augmente, les émissions et la consommation augmentent aussi, mais pas de façon parfaitement régulière.

- **NumberofFloors**  
  Le nombre d’étages a un effet limité, car avoir plus d’étages ne signifie pas toujours consommer ou émettre beaucoup plus.

- **YearBuilt**  
  L’année de construction influence peu les émissions et l’énergie, même si les bâtiments récents semblent parfois un peu plus efficaces.

- **NumberofBuildings**  
  Le nombre de bâtiments a très peu d’impact sur les émissions et la consommation par rapport aux autres variables.


#### 2. Boxplots : Features catégorielles vs Cibles
Distribution des cibles selon différentes catégories.

In [ ]:
features_categorielles = ['BuildingType', 'PrimaryPropertyType', 'Neighborhood']

for cat_feature in features_categorielles:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle(f'Distribution des cibles par {cat_feature}', 
                fontsize=14, fontweight='bold')
    
    for j, cible in enumerate(cibles):
        # Si trop de catégories, garder seulement les top 10
        if df_clean[cat_feature].nunique() > 15:
            top_cats = df_clean[cat_feature].value_counts().head(10).index
            data_subset = df_clean[df_clean[cat_feature].isin(top_cats)]
        else:
            data_subset = df_clean
        
        # Créer le boxplot
        data_subset.boxplot(column=cible, by=cat_feature, ax=axes[j], 
                           patch_artist=True, grid=False)
        axes[j].set_xlabel(cat_feature, fontsize=11)
        axes[j].set_ylabel(cible, fontsize=11)
        axes[j].set_title('')
        axes[j].tick_params(axis='x', rotation=45, labelsize=9)
        axes[j].get_figure().suptitle('')
    
    plt.tight_layout()
    plt.savefig(f'boxplot_{cat_feature}.png', dpi=300, bbox_inches='tight')
    plt.show()

**Observations Boxplot BuildingType** :
- Les **campus** ont la plus grande variabilité en consommation/émissions
- Les bâtiments **NonResidential** standards montrent une distribution plus homogène
- Quelques outliers dans chaque catégorie

**Observations Boxplot PrimaryPropertyType (Top 10)** :
- **Hôpitaux** et **Hôtels** : Consommation élevée et variable
- **Bureaux (Office)** : Distribution large mais médiane modérée
- **Retail/Commercial** : Consommation généralement plus faible
- **Data Centers** (si présents) : Très énergivores

**Observations Boxplot Neighborhood** :
- **Downtown** : Concentration de bâtiments à forte consommation (grands immeubles)
- Quartiers résidentiels : Consommation généralement plus faible
- La variabilité intra-quartier est importante

#### Pairplot : Vue d'ensemble
Matrice de scatterplots croisant 5 variables (3 features + 2 cibles) sur un échantillon de 500 bâtiments.

In [ ]:
features_pairplot = [
    'PropertyGFABuilding(s)',
    'NumberofFloors',
    'YearBuilt',
    'TotalGHGEmissions',
    'SiteEnergyUse(kBtu)'
]

# Échantillonner pour accélérer (optionnel)
sample_size = min(500, len(df_clean))
df_sample = df_clean[features_pairplot].sample(n=sample_size, random_state=42)

# Créer le pairplot
pairplot = sns.pairplot(df_sample,
                        diag_kind='hist',
                        plot_kws={'alpha': 0.6, 's': 20},
                        diag_kws={'bins': 30, 'edgecolor': 'black'})

pairplot.fig.suptitle('Pairplot - Features structurelles et cibles\n(échantillon de 500 bâtiments)',
                      y=1.01, fontsize=14, fontweight='bold')

plt.savefig('pairplot_features_targets.png', dpi=200, bbox_inches='tight')
plt.show()

**Observations** :
- ✅ **Relation linéaire claire** entre les deux cibles (r ≈ 0.7)
- ✅ **PropertyGFABuilding(s)** est corrélé positivement avec les deux cibles
- 📊 Les distributions en diagonale montrent :
  - GHG et Energy : distributions asymétriques à droite
  - Surface : distribution concentrée sur les petites/moyennes surfaces
  - YearBuilt : pic dans les années 1960-1980

### Graphiques supplémentaires

In [ ]:
# 4.1 Top types de propriétés par émissions/consommation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Émissions moyennes par type
property_ghg = df_clean.groupby('PrimaryPropertyType')['TotalGHGEmissions'].mean().sort_values(ascending=False).head(10)
axes[0].barh(range(len(property_ghg)), property_ghg.values, color='coral')
axes[0].set_yticks(range(len(property_ghg)))
axes[0].set_yticklabels(property_ghg.index, fontsize=10)
axes[0].set_xlabel('TotalGHGEmissions moyenne', fontsize=11)
axes[0].set_title('Top 10 types de propriétés\npar émissions GES moyennes', 
                 fontsize=12, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Consommation moyenne par type
property_energy = df_clean.groupby('PrimaryPropertyType')['SiteEnergyUse(kBtu)'].mean().sort_values(ascending=False).head(10)
axes[1].barh(range(len(property_energy)), property_energy.values, color='steelblue')
axes[1].set_yticks(range(len(property_energy)))
axes[1].set_yticklabels(property_energy.index, fontsize=10)
axes[1].set_xlabel('SiteEnergyUse(kBtu) moyenne', fontsize=11)
axes[1].set_title('Top 10 types de propriétés\npar consommation énergétique moyenne',
                 fontsize=12, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('top_property_types.png', dpi=300, bbox_inches='tight')
plt.show()


#### Relation entre les deux cibles

In [ ]:

fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(df_clean['SiteEnergyUse(kBtu)'],
                    df_clean['TotalGHGEmissions'],
                    c=df_clean['PropertyGFABuilding(s)'],
                    cmap='viridis',
                    alpha=0.6,
                    s=30)

ax.set_xlabel('SiteEnergyUse(kBtu)', fontsize=12)
ax.set_ylabel('TotalGHGEmissions', fontsize=12)
ax.set_title('Relation entre consommation énergétique et émissions GES\n(couleur = surface du bâtiment)',
            fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

# Corrélation
corr = df_clean['SiteEnergyUse(kBtu)'].corr(df_clean['TotalGHGEmissions'])
ax.text(0.05, 0.95, f'Corrélation : r = {corr:.3f}',
       transform=ax.transAxes,
       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
       verticalalignment='top', fontsize=12, fontweight='bold')

# Colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Surface du bâtiment (sq ft)', fontsize=11)

plt.tight_layout()
plt.savefig('relation_targets.png', dpi=300, bbox_inches='tight')
plt.show()

**Observations** :
- **Corrélation forte** entre SiteEnergyUse et TotalGHGEmissions (r ≈ 0.7-0.8)
- Les **bâtiments plus grands** (couleur plus claire) ont tendance à être en haut à droite
- Relation quasi-linéaire → prédire l'une des cibles aide à prédire l'autre
- Quelques bâtiments avec **émissions disproportionnées** par rapport à leur consommation (probablement forte utilisation de gaz naturel)

## 🔍 Insights principaux

In [ ]:
print("\n" + "="*80)
print("RÉSUMÉ DES OBSERVATIONS")
print("="*80)

# Calculer les corrélations
numeric_features = [col for col in df_clean.select_dtypes(include=[np.number]).columns 
                   if col not in cibles]

print("\nTop 5 features corrélées avec TotalGHGEmissions :")
corr_ghg = df_clean[numeric_features + ['TotalGHGEmissions']].corr()['TotalGHGEmissions'].sort_values(ascending=False)[1:6]
for i, (feat, corr) in enumerate(corr_ghg.items(), 1):
    print(f"  {i}. {feat}: {corr:.3f}")

print("\nTop 5 features corrélées avec SiteEnergyUse(kBtu) :")
corr_energy = df_clean[numeric_features + ['SiteEnergyUse(kBtu)']].corr()['SiteEnergyUse(kBtu)'].sort_values(ascending=False)[1:6]
for i, (feat, corr) in enumerate(corr_energy.items(), 1):
    print(f"  {i}. {feat}: {corr:.3f}")



### Features les plus importantes pour TotalGHGEmissions
1. **NaturalGas(kBtu)** : r = 0.907 ⭐
2. **GHGEmissionsIntensity** : r = 0.648
3. **SiteEUI(kBtu/sf)** : r = 0.549
4. **Electricity(kBtu)** : r = 0.416
5. **PropertyGFABuilding(s)** : r = 0.382

### Features les plus importantes pour SiteEnergyUse(kBtu)
1. **Electricity(kBtu)** : r = 0.922 ⭐
2. **PropertyGFATotal** : r = 0.682
3. **PropertyGFABuilding(s)** : r = 0.652
4. **LargestPropertyUseTypeGFA** : r = 0.637
5. **SourceEUI(kBtu/sf)** : r = 0.550

### ⚠️ Important : Data Leakage à éviter !

**Variables à NE PAS utiliser dans la modélisation** (ce sont des composantes ou dérivées des cibles) :
- ❌ `NaturalGas(kBtu)` → composante de SiteEnergyUse
- ❌ `Electricity(kBtu)` → composante de SiteEnergyUse
- ❌ `SteamUse(kBtu)` → composante de SiteEnergyUse
- ❌ `SiteEUI(kBtu/sf)` → dérivée de SiteEnergyUse
- ❌ `SourceEUI(kBtu/sf)` → dérivée de SiteEnergyUse
- ❌ `GHGEmissionsIntensity` → dérivée de TotalGHGEmissions

In [ ]:
df.to_csv('data/cleaned_data.csv', index=False)